# Bibliothèques

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rd
import numpy.linalg as nla
import pandas as pnd
import sklearn.covariance as sklcov
import sklearn.cluster as sklclu
import sklearn.metrics.cluster as sklmc
import itertools as itt
import scipy.cluster.hierarchy as spch

In [ ]:
'''====================================
    Ensemble de modules créés par Pierre
    ====================================
    
    ======================================= ============================================================
    Nécessite les bibliothèques suivantes :
    --------------------------------------- ------------------------------------------------------------
    numpy
    matplotlib
    numpy.random
    numpy.linalg
    pandas
    sklearn.covariance
    sklearn.cluster
    sklearn.metrics.cluster
    itertools
    =============================== ====================================================================
        
    =============================== ====================================================================
    Contient les modules suivants :
    ------------------------------- --------------------------------------------------------------------
    maybe_useful                    Fonctions peut-être utiles.
    for_clus                        Fonctions utiles pour le clustering.
    single_fa                       Fonctions d'estimation pour modèles simples.
    clus_funs                       Fonctions de clustering.
    data_rnr                        Fonctions pour reconstruire et représenter les données.
    for_fs                          Fonctions utiles pour la sélection de variables.
    mixed_fa                        Fonctions d'estimation pour mixtures de modèles.
    simulate                        Fonctions pour simuler des paramètres et des données.
    fs_funs                         Fonctions de sélection de variables.
    =============================== ====================================================================
'''

# Fonctions peut-être utiles (MBU)

In [ ]:
'''==========================
    Fonctions peut-être utiles
    ==========================
    
    ================================== ====================================================================
    Contient les fonctions suivantes :
    ---------------------------------- --------------------------------------------------------------------
    norme                              Norme L2 d'un vecteur.
    angle_oriente                      Angle orienté entre un vecteur de taille 2 et le vecteur (1,0).
    rotation                           Matrice de rotation de taille (2,2) associée à un angle.
    erreur_Z                           Distance en norme L2 entre deux matrices à 2 colonnes,
                                       après avoir rotaté l'une des deux.
    orthogonalize                      Orthogonalisation d'une matrice injective.
    normalize                          Normalisation des colonnes d'une matrice.
    trace                              Trace d'une matrice carrée.
    cov_emp                            Matrice de covariance empirique de vecteurs aléatoires i.i.d.
    L_knee                             Emplacement du "genou" d'un vecteur, si celui-ci se trouve à gauche.
    R_knee                             Emplacement du "genou" d'un vecteur, si celui-ci se trouve à droite.
    R_elbow                            Emplacement du "coude" d'un vecteur, si celui-ci se trouve à droite.
    L_elbow                            Emplacement du "coude" d'un vecteur, si celui-ci se trouve à gauche.
    ================================== ====================================================================
'''

In [2]:
def norme(x):
    '''Norme L2 du vecteur x.
    
    Paramètres
    ----------
    x : 1-D ndarray,
        Vecteur dont on veut calculer la norme.
    
    Renvois
    -------
    norme_x : float,
        Norme L2 du vecteur x.
    '''
    return np.sqrt(x@x)

def angle_oriente(x):
    '''Angle orienté entre vecteur x et le vecteur (1,0).
    
    Paramètres
    ----------
    x : 1-D ndarray,
        Vecteur non-nul de longueur 2 dont on veut calculer l'angle orienté avec le vecteur (1,0).
    
    Renvois
    -------
    alpha : float,
        Angle orienté entre vecteur x et le vecteur (1,0).
    '''
    
    nx = x/norme(x)
    if nx[1] >= 0:
        return np.arccos(nx[0])
    else :
        return -np.arccos(nx[0])

def rotation(a):
    '''Matrice de rotation de taille (2,2) associée à l'angle a.
    
    Paramètres
    ----------
    a : float,
        Angle dont on veut la matrice de rotation de taille (2,2) associée.
    
    Renvois
    -------
    R : 2-D ndarray,
        Matrice de rotation de taille (2,2) associée à l'angle a.
    '''
    R = np.array([[np.cos(a), np.sin(a)],[-np.sin(a), np.cos(a)]])
    return R

def erreur_Z(Z1,Z2,a,sym=False):
    '''Distance en norme L2 entre la matrice Z1, et le produit de la matrice de rotation associée à l'angle a par la matrice Z2.
    
    Paramètres
    ----------
    Z1 : 2-D ndarray,
        Matrice de taille (N,2).
    
    Z2 : 2-D ndarray,
        Matrice de taille (N,2).
        
    a : float,
        Angle avec lequel on veut rotater les lignes de la matrices Z2.
    
    sym : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, reflète orthogonalement les vecteurs de Z2 sur l'axe des y avant de les rotater.
        Mis sur False par défaut.
        
    Renvois
    -------
    dist : float,
        Distance en norme L2 entre la matrice Z1, et le produit de la matrice de rotation associée à l'angle a par la matrice Z2.
    '''
    S = np.array([[1,0],[0,1-2*float(sym)]])
    R = np.transpose(rotation(a)@S)
    RZ2 = Z2@R
    return np.sum((Z1-RZ2)**2)

def orthogonalize(W):
    '''Matrice de même taille que W dont les colonnes sont orthogonales.
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice injective à orthogonaliser.
    
    Renvois
    -------
    W_orth : 2-D ndarray,
        Matrice de même taille que W dont les colonnes sont orthogonales.
    '''
    R2 = np.transpose(W)@W
    SpR2, P = nla.eig(R2)
    R = np.real(P @ np.diag(1/np.sqrt(SpR2)) @ nla.inv(P))
    W_2 = W @ R
    return W_2

def normalize(P):
    '''Matrice de même ensemble d'images que P, mais dont les colonnes sont de norme 1.
    
    Paramètres
    ----------
    P : 2-D ndarray,
        Matrice injective à normaliser.
    
    Renvois
    -------
    P_orth : 2-D ndarray,
        Matrice de même ensemble d'images que P, mais dont les colonnes sont de norme 1.
    '''
    P_norms = np.sqrt(np.sum(P**2,axis=0))
    D_P = np.diag(1/P_norms)
    P2 = P@D_P
    return P2

def trace(A):
    '''Trace de la matrice A.
    
    Paramètres
    ----------
    A : 2-D ndarray,
        Matrice carrée dont on veut calculer la trace.
    
    Renvois
    -------
    tr_A : float,
        Trace de la matrice A.
    '''
    N1,N2 = np.shape(A)
    if N1 != N2 :
        print('Erreur de dimensions sur A')
    else :
        return np.sum([A[n][n] for n in range(N1)])

def cov_emp(Y):
    '''Matrice de covariance empirique des vecteurs de la matrice Y.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice dont les lignes sont des vecteurs aléatoires, supposément indépendants et de même loi, dont on veut calculer la covariance.
    
    Renvois
    -------
    S : 2-D ndarray,
        Matrice de covariance empirique des vecteurs de la matrice Y.
    '''
    N,D = np.shape(Y)
    mu_Y = np.mean(Y,axis=0)
    Yc = Y - mu_Y
    S = 1/N * np.transpose(Yc) @ Yc
    return S

def L_knee(y,x=None,alpha=1.0):
    '''Emplacement du "genou" du vecteur y, si celui-ci se trouve à gauche.
    
    Paramètres
    ----------
    y : 1-D ndarray,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "genou".
    
    x : 1-D ndarray, optional,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "genou".
        Mis par défaut sur np.range(0,N) ou N est la taille de y.
    
    alpha : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "genou".
        Plus alpha est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "genou" sera grande, et donc plus le "genou" sera pris à droite.
        Mis sur 1.0 par défaut.
        
    Renvois
    -------
    n_knee : int,
        Emplacement du "genou" du vecteur y, si celui-ci se trouve à gauche.
    '''
    N = len(y)
    ord_y = np.sort(y)
    
    if type(x) == type(None) or len(x) != N:
        x = np.arange(0,N)
    ord_x = np.sort(x)
    
    #Renormalisation
    vert_length = ord_y[-1]-ord_y[0]
    horz_length = ord_x[-1]-ord_x[0]
    x1_list = np.array([(x-ord_x[0])/horz_length for x in ord_x])
    y1_list = np.array([(y-ord_y[0])/vert_length for y in ord_y])
    z1_list = alpha*y1_list - x1_list
    
    n_star = int(np.argmax(z1_list))
    return n_star

def R_knee(y,x=None,alpha=1.0):
    '''Emplacement du "genou" du vecteur y, si celui-ci se trouve à droite.
    
    Paramètres
    ----------
    y : 1-D ndarray,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "genou".
    
    x : 1-D ndarray, optional,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "genou".
        Mis par défaut sur np.range(0,N) ou N est la taille de y.
    
    alpha : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "genou".
        Plus alpha est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "genou" sera grande, et donc plus le "genou" sera pris à gauche.
        Mis sur 1.0 par défaut.
        
    Renvois
    -------
    n_knee : int,
        Emplacement du "genou" du vecteur y, si celui-ci se trouve à droite.
    '''    
    N = len(y)
    return N - L_knee(y,x,alpha)

def R_elbow(y,x=None,alpha=1.0):
    '''Emplacement du "coude" du vecteur y, si celui-ci se trouve à droite.
    
    Paramètres
    ----------
    y : 1-D ndarray,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "coude".
    
    x : 1-D ndarray, optional,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "coude".
        Mis par défaut sur np.range(0,N) ou N est la taille de y.
    
    alpha : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "coude".
        Plus alpha est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "coude" sera grande, et donc plus le "coude" sera pris à gauche.
        Mis sur 1.0 par défaut.
        
    Renvois
    -------
    n_elbow : int,
        Emplacement du "coude" du vecteur y, si celui-ci se trouve à droite.
    '''
    N = len(y)
    ord_y = np.sort(y)
    
    if type(x) == type(None) or len(x) != N:
        x = np.arange(0,N)
    ord_x = np.sort(x)
    
    #Renormalisation
    vert_length = ord_y[-1]-ord_y[0]
    horz_length = ord_x[-1]-ord_x[0]
    x1_list = np.array([(x-ord_x[0])/horz_length for x in ord_x])
    y1_list = np.array([(y-ord_y[0])/vert_length for y in ord_y])
    z1_list = x1_list - alpha*y1_list
    
    n_star = int(np.argmax(z1_list))
    return n_star

def L_elbow(y,x=None,alpha=1.0):
    '''Emplacement du "coude" du vecteur y, si celui-ci se trouve à gauche.
    
    Paramètres
    ----------
    y : 1-D ndarray,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "coude".
    
    x : 1-D ndarray, optional,
        Vecteur d'ordonnées dont on veut calculer l'emplacement du "coude".
        Mis par défaut sur np.range(0,N) ou N est la taille de y.
    
    alpha : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "coude".
        Plus alpha est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "coude" sera grande, et donc plus le "coude" sera pris à droite.
        Mis sur 1.0 par défaut.
        
    Renvois
    -------
    n_elbow : int,
        Emplacement du "coude" du vecteur y, si celui-ci se trouve à gauche.
    '''
    N = len(y)
    return N - R_elbow(y,x,alpha)

# Fonctions utiles pour le clustering (UFC)

In [ ]:
'''===================================
    Fonctions utiles pour le clustering
    ===================================
    
    ================================== =========================================================================
    Contient les fonctions suivantes :
    ---------------------------------- -------------------------------------------------------------------------
    d_SL                               Distance en Single-Linkage entre 2 clusters.
    d_CL                               Distance en Complete-Linkage entre 2 clusters.
    d_AL                               Distance en Average-Linkage entre 2 clusters.
    d_L2_Ward                          Distance de Ward entre 2 clusters.
    dissim_L2                          Matrice de dissimilarité en distance L2 entre les lignes d'une matrice.
    condense                           Condensation en vecteur d'une matrice symétrique.
    tri                                Tri des lignes d'une matrice selon un clustering donné.
    omegate                            Transformation d'une liste de clusters en un vecteur d'entiers.
    matrixage                          Transformation d'un vecteur contenant des numéros de clusters
                                       en une matrice contenant des 0 et des 1.
    occurences                         Nombre d'occurences de chaque coefficient dans un vecteur d'entiers.
    perm_opt                           Permutation optimale pour faire correspondre ensemble
                                       deux vecteurs d'entiers de même taille.
    Dist_CP                            Distance L2 minimale d'un vecteur à une liste de vecteurs de même taille.
    sil_coeff                          Coefficient silhouette d'un vecteur d'une matrice d'observations
                                       pour un clustering donné.
    sil_score                          Score silhouette d'une matrice observations pour un clustering donné.
    distorsion                         Distorsion d'une matrice d'observations pour un clustering donné.
    Lap                                Laplacienne normalisée d'une matrice de similarité.
    ARS                                Adjusted Rand Score entre deux vecteurs d'entiers de même taille.
    ================================== =========================================================================
'''

In [3]:
#Single-linkage
def d_SL(Pdm,G,H):
    '''Distance en Single-Linkage entre les clusters G et H.
    
    Paramètres
    ----------
    Pdm : 2-D ndarray,
        Matrice symétrique d'ordre N de dissimilarité dont chaque coefficient est un réel positif quantifiant la dissimilarité entre deux individus (une distance par exemple). 
    
    G : list,
        Liste des numéros des individus du cluster G, devant être compris entre 0 et N-1.
        
    H : list,
        Liste des numéros des individus du cluster H, devant être compris entre 0 et N-1.
    
    Renvois
    -------
    dist : float,
        Distance en Single-Linkage entre les clusters G et H.
    '''
    if np.any(Pdm-np.transpose(Pdm)) or np.any(Pdm<0):
        print("Pdm n'est pas une matrice de dissimilarité")
    else :
        N,N1 = np.shape(Pdm)
        dist_tab = np.array([[Pdm[i][j] for j in H] for i in G])
        return np.min(dist_tab)

#Complete-linkage
def d_CL(Pdm,G,H):
    '''Distance en Complete-Linkage entre les clusters G et H.
    
    Paramètres
    ----------
    Pdm : 2-D ndarray,
        Matrice symétrique d'ordre N de dissimilarité dont chaque coefficient est un réel positif quantifiant la dissimilarité entre deux individus (une distance par exemple). 
    
    G : list,
        Liste des numéros des individus du cluster G, devant être compris entre 0 et N-1.
        
    H : list,
        Liste des numéros des individus du cluster H, devant être compris entre 0 et N-1.
    
    Renvois
    -------
    dist : float,
        Distance en Complete-Linkage entre les clusters G et H.
    '''
    if np.any(Pdm-np.transpose(Pdm)) or np.any(Pdm<0):
        print("Pdm n'est pas une matrice de dissimilarité")
    else :
        N,N1 = np.shape(Pdm)
        dist_tab = np.array([[Pdm[i][j] for j in H] for i in G])
        return np.max(dist_tab)

#Average-linkage
def d_AL(Pdm,G,H):
    '''Distance en Average-Linkage entre les clusters G et H.
    
    Paramètres
    ----------
    Pdm : 2-D ndarray,
        Matrice symétrique d'ordre N de dissimilarité dont chaque coefficient est un réel positif quantifiant la dissimilarité entre deux individus (une distance par exemple). 
    
    G : list,
        Liste des numéros des individus du cluster G, devant être compris entre 0 et N-1.
        
    H : list,
        Liste des numéros des individus du cluster H, devant être compris entre 0 et N-1.
    
    Renvois
    -------
    dist : float,
        Distance en Average-Linkage entre les clusters G et H.
    '''
    if np.any(Pdm-np.transpose(Pdm)) or np.any(Pdm<0):
        print("Pdm n'est pas une matrice de dissimilarité")
    else :
        N,N1 = np.shape(Pdm)
        dist_tab = np.array([[Pdm[i][j] for j in H] for i in G])
        return np.mean(dist_tab)

#Distance Ward
def d_L2_Ward(X,G,H):
    '''Distance de Ward entre les clusters G et H.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les individus à clusteriser.
    
    G : list,
        Liste des numéros des individus du cluster G, devant être compris entre 0 et N-1.
        
    H : list,
        Liste des numéros des individus du cluster H, devant être compris entre 0 et N-1.
    
    Renvois
    -------
    dist : float,
        Distance de Ward entre les clusters G et H.
    '''    
    N,D = np.shape(X)
    
    if max(G) >= N or max(H) >= N :
        print("Pas assez d'individus")
    else :
        mu_G = np.mean(np.array([X[n] for n in G]),axis=0)
        mu_H= np.mean(np.array([X[n] for n in H]),axis=0)
        return np.sum((mu_G - mu_H)**2)

#Distance L2
def dissim_L2(X):
    '''Matrice de dissimilarité en distance L2 des lignes de X.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des individus.
    
    Renvois
    -------
    Pdm : 2-D ndarray,
        Matrice carré d'ordre N, dont, pour tous i,j entre 0 et N-1, le coefficient à la i-ème ligne et la j-ième colonne est la distance L2 entre le i-ème et le j-ème vecteur ligne de X.
    '''
    N,D = np.shape(X)
    return np.array([[np.sqrt(np.sum((X[i]-X[j])**2)) for i in range(N)] for j in range(N)])

#Condensation d'une matrice de dissimilarité
def condense(PdM):
    '''Condensation d'une matrice symétrique.
    
    Paramètres
    ----------
    PdM : 2-D ndarray,
        Matrice symétrique d'ordre N.
    
    Renvois
    -------
    conc : 1-D ndarray,
        Concaténation des lignes de la partie triangulaire strictement supérieure de PdM.
    '''
    N,N1 = np.shape(PdM)
    return np.concatenate([PdM[n][n+1:] for n in range(N-1)])

#Tri des vecteurs
def tri(X,omega,K=None):
    '''Tri des vecteurs de X selon omega.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les individus à trier.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    K : int, optional,
        Mis à défaut sur la valeur maximale de omega + 1
        Si renseigné et plus grand que la valeur maximale de omega + 1, rendra des clusters vides.
        Si renseigné et plus petit que la valeur maximale de omega + 1, ne prendra pas en compte les individus dont le numéro de cluster est plus grand que K-1
    
    Renvois
    -------
    tri_X : list of ndarray,
        Liste à K éléments dont chaque élément est la matrice de taille (N_k,D) dont les lignes sont les individus d'un même cluster.
    '''
    N = len(omega)
    if type(K) == type(None):
        K = int(max(omega) + 1)
        
    for k in range(K):
        if k not in omega :
            print(k)
            print("tri : Clusters vides")
    
    tri_X = [np.array([]) for k in range(K)]
    
    for k in range(K):
        tri_X[k] = np.array([X[n] for n in range(N) if omega[n]==k])
    
    return tri_X

def omegate(clusters):
    '''Transformation d'une liste de clusters en un vecteur d'entiers.
    
    Paramètres
    ----------
    
    clusters : list of list,
        Liste de K listes, où, pour tout k entre 0 et K-1, le k-ième liste contient les numéros des individus appartenant au k-ième cluster.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    K = len(clusters)
    N = max([max(clus) for clus in clusters]) + 1
    omega = np.zeros(N)
    
    for k in range(K) :
        for n in clusters[k] :
            omega[n] = k
    
    return omega.astype(int)

def matrixage(omega):
    '''Transformation d'un vecteur contenant des numéros de clusters en une matrice avec des 0 et des 1.
    
    Paramètres
    ----------
    omega : 1-D ndarray,
        Vecteur d'entiers de taille N et de valeur maximale K-1.
        
    Renvois
    -------
    O : 2-D ndarray,
        Matrice de taille (N,K), dont, pour tout n entre 0 et N-1 et tout k entre 0 et K-1, le coefficient à la n-ième ligne et la k-ième colonne vaut 1 si le n-ième coefficient de omega vaut k et 0 sinon.
    '''
    N = len(omega)
    K = int(np.max(omega) + 1)
    O = np.array([[int(omega[n] == k) for k in range(K)] for n in range(N)])
    
    return O

#Occurences
def occurences(omega):
    '''Nombres d'occurences de chaque coefficient d'un vecteur d'entiers.
    
    Paramètres
    ----------
    omega : 1-D ndarray,
        Vecteur d'entiers de taille N et de valeur maximale K-1.
        
    Renvois
    -------
    occ : 1-D ndarray,
        Vecteur de taille K, dont, pour tout k entre K-1, le k-ième coefficient de omega est le nombre d'occurences de k dans omega.
    '''
    N = len(omega)
    K = int(np.max(omega)) + 1
    
    occur = np.zeros(K)
    for n in range(N):
        occur[omega[n]] += 1
    
    return occur.astype(int)

def perm_opt(omega1,omega2):
    '''Permutation optimale pour faire correspondre ensemble deux vecteurs d'entiers de même taille.
    
    Paramètres
    ----------
    omega1 : 1-D ndarray,
        Vecteur d'entiers de taille N et de valeur maximale K-1.
        
    omega2 : 1-D ndarray,
        Vecteur d'entiers de taille N et de valeur maximale K-1.
        
    Renvois
    -------
    s : 1-D ndarray,
        Permutation pour laquelle le nombre de coefficients différant entre s(omega1) et omega2 est minimal.
    '''
    N1 = len(omega1)
    N2 = len(omega2)
    occ1 = occurences(omega1)
    occ2 = occurences(omega2)
    K1 = len(occ1)
    K2 = len(occ2)
    
    if N1 != N2 or K1 != K2 or np.any(occ1==0) or np.any(occ2==0):
        print("Les omegas ne correspondent pas")
    else:
        N = N1
        K = K1
        
        O1 = matrixage(omega1)
        O2 = matrixage(omega2)
        R_occ = np.transpose(O1)@O2
        R = np.diag(1/occ1) @ R_occ
        
        sure=[]
        unsure=list(np.arange(K))
        taken=[]
        untaken=list(np.arange(K))
        
        s_sure = -np.ones(K)
        
        for k in range(K):
            s_k = int(np.argmax(R[k]))
            if k == int(np.argmax(R[:,s_k])):
                sure.append(k)
                unsure.remove(k)
                taken.append(s_k)
                untaken.remove(s_k)
                s_sure[k] = s_k
        
        nb_unsure = len(unsure)
        
        errs = []
        perms = list(itt.permutations(range(nb_unsure)))
        for perm in perms :
            s_test = s_sure
            for i in range(nb_unsure):
                j = perm[i]
                s_test[unsure[i]] = untaken[j]
            s_omega1 = np.array([s_test[o] for o in omega1]).astype(int)
            err = np.sum(((s_omega1 - omega2).astype(bool)).astype(float))
            errs.append(err)
        
        ind_opt = int(np.argmin(np.array(errs)))
        us_opt = perms[ind_opt]
        s_opt = s_sure
        for i in range(nb_unsure):
            j = perm[i]
            s_opt[unsure[i]] = untaken[j]
        
        return s_opt.astype(int)

#Distance to closest point
def Dist_CP(x,M):
    '''Distance L2 minimale d'un vecteur x à une liste M de vecteurs de même taille que x.
    
    Paramètres
    ----------
    x : 1-D ndarray,
        Vecteur de taille D.
        
    M : 2-D ndarray,
        Matrice de taille (t,D), dont les lignes sont les vecteurs dont il faut calculer la distance à x
    
    Renvois
    -------
    dist : float,
        Distance L2 de x au vecteur ligne de M qui lui est le plus proche.
    '''
    t,D = np.shape(M)
    D1 = len(x)
    if D != D1:
        print("x et les moyennes n'ont pas même dimensions")
    else :
        return min([np.sum((x-M[k])**2) for k in range(t)])

#Silhouette coefficient
def sil_coeff(n,X,omega):
    '''Coefficient silhouette du n-ième individu des observations X pour le clustering induit par omega.
    
    Paramètres
    ----------
    n : int,
        Numéro de l'individu dont on veut le coefficient silhouette. Doit être compris entre 0 et N-1.
        
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Renvois
    -------
    coeff : float,
        Coefficient silhouette du n-ième individu des observations X pour le clustering donné par omega.
    '''
    x = X[n]
    N = len(X)
    K = int(max(omega)+1)
    
    M = np.array([np.mean(np.array([X[j] for j in range(n) if omega[j]==k])) for k in range(K)])
    
    k_star = omega[n]
    dists = np.array([np.sum((x-M[k])**2) for k in range(K)])
    dists[k_star] += np.max(dists) + 1
    k_prime = np.argmin(dists)
    
    a = np.mean(np.array([np.sum((x-X[j])**2) for j in range(N) if omega[j] == k_star]))
    b = np.mean(np.array([np.sum((x-X[j])**2) for j in range(N) if omega[j] == k_prime]))
    
    return (b-a)/(max(a,b))

#Silhouette score
def sil_score(X,omega):
    '''Score silhouette des observations X pour le clustering induit par omega.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Renvois
    -------
    score : float,
        Score silhouette des observations X pour le clustering donné par omega.
    '''
    N = len(X)
    return np.mean([sil_coeff(n,X,omega) for n in range(N)])

#Distorsion
def distorsion(X,omega):
    '''Distorsion des observations X pour le clustering induit par omega.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Renvois
    -------
    dist : float,
        Somme des erreurs en distance L2 de chaque individu au centre de son cluster.
    '''
    tri_X = tri(X,omega)
    return np.sum(np.array([np.sum(np.var(x,axis=0)) for x in tri_X]))

def Lap(Psm):
    '''Laplacienne normalisée d'une matrice de similarité
    
    Paramètres
    ----------
    Psm : 2-D ndarray,
        Matrice de similarité pairwise symétrique d'ordre N, dont les coefficients sont positifs et sont d'autant plus élevés que l'individu en abscisse et celui en ordonnée sont proches.
    
    Renvois
    -------
    Lapsym : 2-D ndarray,
        Matrice Laplacienne normalisée de Psm
    '''
    N,N1 = np.shape(Psm)
    
    if np.any(Psm-np.transpose(Psm)) or np.any(Psm<0):
        print("Psm n'est pas une matrice de poids symétrique")
    else :
        vec_D = np.sum(Psm, axis=0)
        rinv_D = np.diag(1/np.sqrt(vec_D))
        
        L = np.eye(N) - rinv_D @ Psm @ rinv_D
        
        return L

def ARS(omega1,omega2):
    '''Adjusted Rand Score entre deux vecteurs d'entiers de même taille.
    
    Paramètres
    ----------
    omega1 : 1-D ndarray,
        Vecteur d'entiers de taille N.
        
    omega2 : 1-D ndarray,
        Vecteur d'entiers de taille N.
        
    Renvois
    -------
    ars : float,
        Adjusted Rand Score entre les clusterings induits par omega1 et omega2.
    '''
    return sklmc.adjusted_rand_score(omega1,omega2)

# Fonctions d'estimation, modèles simples (SFA)

In [ ]:
'''===========================================
    Fonctions d'estimation pour modèles simples
    ===========================================
    
    ================================== ========================================================================
    Contient les fonctions suivantes :
    ---------------------------------- ------------------------------------------------------------------------
    L_opt                              Nombre optimal de dimensions latentes, estimé par la "méthode du saut"
    L_opt_2                            Nombre optimal de dimensions latentes, estimé par la "méthode du coude".
    L_opt_3                            Nombre optimal de dimensions latentes, estimé par la "méthode du genou".
    PCA                                Analyse en Composante Principale (Adapté pour le modèle (M.1)).
    bruit                              Estimation de la variance du bruit à partir d'une matrice de covariance.
    PPCA_EM                            Analyse en Composante Principale Probabiliste, algorithme E-M
                                       (Adapté pour le modèle (M.1)).
    PPCA                               Analyse en Composante Principale Probabiliste, algorithme direct
                                       (Adapté pour le modèle (M.1)).
    ML_RCA                             Analyse en Composante Résiduelle, méthode itérative
                                       (Adapté pour le modèle (M.2)).
    MLE_Gauss                          Estimation de la moyenne et de la variance d'un N-échantillon Gaussien
                                       multivarié de covariance isotrope.
    ================================== ========================================================================
'''

In [2]:
def L_opt(S,L_min=1,detail=False):
    '''Nombre optimal de dimensions latentes, estimé par la "méthode du saut".
    
    Paramètres
    ----------
    S : 2-D ndarray,
        Matrice de covariance empirique (symétrique positive d'ordre D) à diagonaliser.
        
    L_min : int, optional,
        Nombre de dimensions minimal à renvoyer.
        Mis sur 1 par défaut.
        
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, détaille graphiquement le choix du nombre de dimensions latentes.
        
        Le coefficient d'incertitude donné est égal à log(total_span/taken_span)/log(D) où
            - total_span est la différence entre la plus grande et la plus petite valeur propre de S.
            - taken_span est la différence entre la L-ième et la (L+1)-ème plus grande valeur propre de S.
        
    Renvois
    -------
    L : int,
        Nombre de dimensions latentes pour la matrice de covariance S minimisant le coefficient d'incertitude.
    '''
    D1,D2 = np.shape(S)
    if D1 != D2:
        print('Erreur de dimensions sur S')
    else :
        D = D1
        SpS,P = nla.eig(S)
        
        ordre = np.sort(SpS)
        if L_min <= 1:
            diff_ordre = ordre[1:] - ordre[:-1]
            L_star = D - int(np.argmax(diff_ordre)) - 1
        else :
            diff_ordre = ordre[1:1-L_min] - ordre[:-L_min]
            L_star = D - int(np.argmax(diff_ordre)) - 1
        
        total_span = ordre[-1]-ordre[0]
        taken_span = ordre[D-L_star]-ordre[D-L_star-1]
        
        coeff_incert = np.log(total_span/taken_span)/np.log(D)
        
        if detail :
            plt.figure()
            plt.step(np.arange(D),ordre)
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],ordre[D-L_star]])
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],0],'--',label='$D-L_{star}$')
            plt.legend()
            plt.title('Spectre ordonné de la matrice de covariance')
            plt.show()
            
            print("Le coefficient d'incertitude est de",coeff_incert)
        
        return L_star

def L_opt_2(S,beta=0.5,L_min=1,detail=False):
    '''Nombre optimal de dimensions latentes, estimé par la "méthode du coude".
    
    Paramètres
    ----------
    S : 2-D ndarray,
        Matrice de covariance empirique (symétrique positive d'ordre D) à diagonaliser.
        
    beta : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "coude".
        Plus beta est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "coude" sera grande, et donc plus la valeur de L renvoyée sera élevée.
        Mis sur 0.5 par défaut.
        
    L_min : int, optional,
        Nombre de dimensions minimal à renvoyer.
        Mis sur 1 par défaut.
        
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, détaille graphiquement le choix du nombre de dimensions latentes.
        
        Le coefficient d'incertitude donné est égal à log(total_span/taken_span)/log(D) où
            - total_span est la différence entre la plus grande et la plus petite valeur propre de S.
            - taken_span est la différence entre la L-ième et la (L+1)-ème plus grande valeur propre de S.
        
    Renvois
    -------
    L : int,
        Nombre de dimensions latentes correspondant à l'emplacement du coude dans le spectre ordonné de S.
    '''
    D1,D2 = np.shape(S)
    if D1 != D2:
        print('Erreur de dimensions sur S')
    else :
        D = D1
        SpS,P = nla.eig(S)
        
        ordre = np.sort(SpS)
        if L_min <= 1:
            L_star = D - mbu.R_elbow(ordre,beta) - 1
        else :
            L_star = D - mbu.R_elbow(ordre[:1-L_min],beta) - 1
        
        total_span = ordre[-1]-ordre[0]
        taken_span = ordre[D-L_star]-ordre[D-L_star-1]
        
        coeff_incert = np.log(total_span/taken_span)/np.log(D)
        
        if detail :
            plt.figure()
            plt.step(np.arange(D),ordre)
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],ordre[D-L_star]],label='$D-L_{star}$')
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],0],'--',label='$D-L_{star}$')
            plt.legend()
            plt.title('Spectre ordonné de la matrice de covariance')
            plt.show()
            
            print("Le coefficient d'incertitude est de",coeff_incert)
        
        return L_star

def L_opt_3(S,beta=0.5,L_min=1,detail=False):
    '''Nombre optimal de dimensions latentes, estimé par la "méthode du genou".
    
    Paramètres
    ----------
    S : 2-D ndarray,
        Matrice de covariance empirique (symétrique positive d'ordre D) à diagonaliser.
        
    beta : float, optional,
        Coefficient pour ajuster la part d'importance des abscisses et des ordonnées dans le calcul de l'emplacement du "genou".
        Plus beta est grand, plus la part d'importance des ordonnées dans le calcul de l'emplacement du "genou" sera grande, et donc plus la valeur de L renvoyée sera élevée.
        Mis sur 0.5 par défaut.
        
    L_min : int, optional,
        Nombre de dimensions minimal à renvoyer.
        Mis sur 1 par défaut.
        
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, détaille graphiquement le choix du nombre de dimensions latentes.
        
        Le coefficient d'incertitude donné est égal à log(total_span/taken_span)/log(D) où
            - total_span est la différence entre la plus grande et la plus petite valeur propre de S.
            - taken_span est la différence entre la L-ième et la (L+1)-ème plus grande valeur propre de S.
        
    Renvois
    -------
    L : int,
        Nombre de dimensions latentes correspondant à l'emplacement du genou dans le scree plot induit par S.
    '''
    D1,D2 = np.shape(S)
    if D1 != D2:
        print('Erreur de dimensions sur S')
    else :
        D = D1
        SpS,P = nla.eig(S)
        
        ordre = np.sort(SpS)
        inertie = np.cumsum(ordre)
        
        if L_min <= 1:
            L_star = D - mbu.R_elbow(inertie,beta) - 1
        else :
            L_star = D - mbu.R_elbow(inertie[:1-L_min],beta) - 1
        
        total_span = ordre[-1]-ordre[0]
        taken_span = ordre[D-L_star]-ordre[D-L_star-1]
        
        coeff_incert = np.log(total_span/taken_span)/np.log(D)
        
        if detail :
            plt.figure()
            plt.step(np.arange(D),ordre)
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],ordre[D-L_star]],label='$D-L_{star}$')
            plt.plot([D-L_star-1,D-L_star-1],[ordre[D-L_star-1],0],'--',label='$D-L_{star}$')
            plt.legend()
            plt.title('Spectre ordonné de la matrice de covariance')
            plt.show()
            
            print("Le coefficient d'incertitude est de",coeff_incert)
        
        return L_star

def PCA(Y,L=None):
    '''Analyse en Composante Principale.
    (Adapté pour le modèle (M.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt
        
    Renvois
    -------
    S : 2-D ndarray,
        Matrice de covariance des vecteurs constituant les lignes de Y.
        
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la PCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PCA.
    '''
    N,D = np.shape(Y)
    
    #Centrage de Y
    mu_ML = np.mean(Y,axis=0)
    Yc = np.array([y-mu_ML for y in Y])
    
    #Diagonalisation de S, choix des axes principaux
    S = mbu.cov_emp(Yc)
    
    if type(L) == type(None):
        L = L_opt(S)
    
    SpS, P = nla.eig(S)
    ordre = np.sort(SpS)
    inlist = [k for k in range(D) if SpS[k] in ordre[D-L:]]
    
    #Estimation de W
    P = mbu.normalize(P)
    tW = np.array([P[:,k] for k in inlist])
    W = np.transpose(tW)
    
    #Estimation de Z
    Z = Yc @ W @ nla.inv(tW@W)
    
    return S, W, Z

def bruit(S,L=None):
    '''Estimation de la variance du bruit à partir d'une matrice de covariance.
    
    Paramètres
    ----------
    S : 2-D ndarray,
        Matrice de covariance empirique (symétrique positive d'ordre D) à diagonaliser.
    
    L : int, optional
        Nombre de dimensions latentes supposé.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt
        
    Renvois
    -------
    sigma2 : float,
        Bruit estimé à partir de la matrice de covariance S, égal à la moyenne des D-L plus petites valeurs propres de S.
    '''
    D1,D2 = np.shape(S)
    
    if type(L) == type(None):
        L = L_opt(S)
        
    if D1 != D2 or D1 <= L or D2 <= L :
        print('Erreur de dimension sur S')
        
    else :
        D = D1
        SpS, P = nla.eig(S)
        sigma2 = np.mean(np.sort(SpS)[D-L:])
        return sigma2

def PPCA_EM(Y,L=None,nb_steps=1000,err=0.0,tempo=True):
    '''Analyse en Composante Principale Probabiliste, algorithme E-M.
    (Adapté pour le modèle (M.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt
        
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme.
        Mis sur 1000 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la PPCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA.
        
    sigma2 : float,
        Estimation de la variance du bruit.
    '''
    N,D = np.shape(Y)
    
    #Centrage de Y
    mu_Y = np.mean(Y,axis=0)
    Yc = np.array([y-mu_Y for y in Y])
    
    #Initialisation
    S = mbu.cov_emp(Y)
    
    if type(L) == type(None):
        L = L_opt(S)
    
    #Estimation de sigma²
    sigma2 = bruit(S,L)
    
    #Algorithme E-M
    dist = err+1
    t = 0
    while dist > err and t < nb_steps :
        new_Z = Yc @ W @ nla.inv(np.transpose(W)@W)
        new_W = np.transpose(Yc) @ new_Z @ nla.inv(np.transpose(new_Z) @ new_Z)
        dist = np.sum((Z-new_Z)**2) + np.sum((W-new_W)**2)
        W = new_W
        Z = new_Z
        t += 1
    if tempo :
        print('t = ',t)
    
    return W, Z, sigma2

def PPCA(Y,L=None):
    '''Analyse en Composante Principale, algorithme direct.
    (Adapté pour le modèle (M.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la PCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PCA.
    
    sigma2 : float,
        Estimation de la variance du bruit.
    '''
    N,D = np.shape(Y)
    
    #Centrage de Y
    mu_ML = np.mean(Y,axis=0)
    Yc = np.array([y-mu_ML for y in Y])
    
    #Estimation de sigma²
    S = mbu.cov_emp(Y)    
    if type(L) == type(None):
        L = L_opt(S)
    
    SpS, P = nla.eig(S)
    P = mbu.normalize(P)
    ordre = np.sort(SpS)
    sigma2 = np.mean(ordre[:D-L])
    inlist = [k for k in range(D) if SpS[k] in ordre[D-L:]]
    
    #Estimation de W et Z
    tU_L = np.array([P[:,k] for k in inlist])
    L_L = np.diag(np.array([np.sqrt(SpS[k]-sigma2) for k in inlist]))
    tW = L_L @ tU_L
    W = np.transpose(tW)
    Z = Yc @ W @ nla.inv(tW@W)
    
    return W, Z, sigma2

def ML_RCA(Y,X,L,V=None,nb_steps=100,err=0.0,tempo=True):
    '''Analyse en Composante Résiduelle, méthode itérative.
    (Adapté pour le modèle (M.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int,
        Nombre de dimensions latentes souhaité.
        
    V : 2-D ndarray, optional,
        Matrice de taille (D,C) d'effets fixes donnée en argument initial de l'algorithme itératif.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme.
        Mis sur 1000 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la RCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la RCA.
        
    V_hat : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes estimée par la RCA.
    
    sigma2 : float,
        Estimation de la variance du bruit.
    '''
    N1,D = np.shape(Y)
    N2,C = np.shape(X)
    
    if N1 != N2 :
        print('Erreur de dimensions sur Y et X')
        S, W, Z = PCA(Y,L)
        sigma2 = bruit(S,L)
        return W, Z, sigma2
    else :
        N = N1
    
    #Centrage de Y et X
    mY = np.mean(Y,axis=0)
    mX = np.mean(X,axis=0)
    Yc = np.array([y-mY for y in Y])
    Xc = np.array([x-mX for x in X])
    
    #Initialisation
    
    #Si V est donné
    if type(V) != type(None) :
        
        #Copie de V s'il est donné
        V_hat = V.copy()
        
        #Estimation de Z et W
        W,Z,sigma2 = PPCA(Yc-Xc@np.transpose(V_hat),L)
        
    #Si V n'est pas donné
    else :
        #Initialisation
        
        #Estimation de V
        V_hat = np.transpose(Yc)@Xc @ nla.inv(np.transpose(Xc)@Xc)
        
        #Estimation de Z et W
        W,Z,sigma2 = PPCA(Yc-Xc@np.transpose(V_hat),L)
        
        dist = err+1
        t = 0
        
        #Boucle
        while dist > err and t < nb_steps :
            
            #Estimation de V
            new_V = np.transpose(Yc-Z@np.transpose(W))@Xc @ nla.inv(np.transpose(Xc)@Xc)
            
            #Estimation de Z et W
            new_W,new_Z,sigma2_hat = PPCA(Yc - Xc@np.transpose(new_V),L)
            
            #Vérification
            dist = np.sum((Z@np.transpose(W) - new_Z@np.transpose(new_W))**2) + np.sum((V_hat-new_V)**2)
            t += 1
            
            W = new_W
            Z = new_Z
            V_hat = new_V
            
        if tempo :
            print('t =',t)
    
    return W, Z, V_hat, sigma2

def tXX_estimator(Y,W,sigma2,Lambda):
    
    D,L = np.shape(W)
    D1,D2 = np.shape(Lambda)
    N,D3 = np.shape(Y)
    
    if D != D1 or D != D2 or D != D3 :
        print('Erreur de dimensions sur Lambda, Y ou W')
    else :
        qty_1 = nla.inv(nla.inv(W@np.transpose(W) + sigma2*np.eye(D)) + Lambda)
        qty_2 = np.array([qty_1 @ nla.inv(W@np.transpose(W) + sigma2*np.eye(D)) @ y for y in Y])
        tXX = np.array([qty_1 + np.transpose(np.array([x]))@np.array([x]) for x in qty_2])
        return tXX

def Lambda_estimator(tXX,la):
    
    N,D1,D2 = np.shape(tXX)
    
    if D1 != D2 :
        print('Erreur de dimensions sur tXX')
    else :
        D = D1
        S = np.mean(tXX,axis=0)
        Lambda, st_else = sklcov.graphical_lasso(S,la,max_iter=50)
        return Lambda
        
def W_estimator(Y,Lambda,sigma2,L):
    
    N,D = np.shape(Y)
    D1,D2 = np.shape(Lambda)
    
    if D != D1 or D != D2 :
        print('Erreur de dimensions sur Y ou Lambda')
    else :
        Sigma = nla.inv(Lambda) + sigma2*np.eye(D)
        S_hat = mbu.cov_emp(Y)
        A = nla.inv(Sigma) @ S_hat
        
        #GEP
        SpA, P = nla.eig(A)
        ord_A = np.sort(SpA)
        inlist_A = [k for k in range(D) if SpA[k] in ord_A[D-L:]]
        P_W = mbu.normalize(np.transpose(np.array([P[:,k] for k in inlist_A])))
        D_W = np.diag(np.array([SpA[k] for k in inlist_A]))
        W = Sigma @ P_W
        
        return W

def log_p_LRPSI(Y,W,Lambda,sigma2,la):
    
    N,D = np.shape(Y)
    D1,L = np.shape(W)
    D2,D3 = np.shape(Lambda)
    
    if D != D1 or D!= D2 or D != D3 :
        print('Erreur de dimensions sur Y, W ou Lambda')
    else :
        Cov = W @ np.transpose(W) + nla.inv(Lambda) + sigma2*np.eye(D)
        inv_Cov = nla.inv(Cov)
        qty = np.sum(np.array([-np.log(np.abs(nla.det(Cov))) - 1/2 * np.array([y])@inv_Cov@y - la*np.sum(np.abs(Lambda)) for y in Y]))
        return qty

def EM_RCA_LRPSI(Y,X,L,Lambda=None,la=None,sigma2=None,nb_steps=1000,err=0.0,tempo=True):
    
    N1,D1 = np.shape(Y)
    N2,D2 = np.shape(X)
    
    if N1 != N2 or D1 != D2 :
        print('Erreur de dimensions sur Y et X')
        S, W, Z = PCA(Y,L)
        sigma2 = bruit(S,L)
        return W, Z, sigma2
    else :
        N = N1
        D = D1
    
    #Centrage de Y et X
    mY = np.mean(Y,axis=0)
    mX = np.mean(X,axis=0)
    Yc = np.array([y-mY for y in Y])
    Xc = np.array([x-mX for x in X])
    
    #Initialisation
    
    #Estimation de Lambda s'il n'est pas donné
    if type(Lambda) == type(None):
        covX = mbu.cov_emp(Xc)
        Lambda_hat = nla.inv(covX)
    else :
        Lambda_hat = Lambda.copy()
    
    D1,D2 = np.shape(Lambda_hat)
    if D != D1 or D != D2 :
        print('Erreur de dimensions sur Lambda')
        S, W, Z = PCA(Yc,L)
        sigma2 = bruit(S,L)
        return W, Z, Lambda_hat, sigma2
    
    #Estimation de lambda s'il n'est pas donné
    if type(la) == type(None):
        la_hat = D**2 * 2/mbu.trace(Lambda_hat)
        calc_la = True
    else :
        la_hat = la.copy()
        calc_la = False
     
    #Estimation de sigma2 s'il n'est pas donné
    if type(sigma2) == type(None):
        S,W_hat,Z = PCA(Yc-Xc,L)
        sigma2 = bruit(S,L)
    
    #Boucle EM/RCA pour estimer Z
    gain = err + 1
    t = 0
    W_hat = W_estimator(Yc,Lambda_hat,sigma2,L)
    log_p = log_p_LRPSI(Yc,W_hat,Lambda_hat,sigma2,la_hat)
    
    while gain > err and t < nb_steps :
        
        #E-step :
        new_tXX = tXX_estimator(Yc,W_hat,sigma2,Lambda_hat)
        
        #M-step :
        new_Lambda = Lambda_estimator(new_tXX,la_hat)
        
        #RCA-step :
        new_W = W_estimator(Yc,new_Lambda,sigma2,L)
        
        #Vérification :
        new_log_p = log_p_LRPSI(Yc,new_W,new_Lambda,sigma2,la_hat)
        gain = new_log_p - log_p
        
        if gain >= 0 :
            Lambda_hat = new_Lambda
            W_hat = new_W
            if calc_la :
                la_hat = D**2 * 2/mbu.trace(Lambda_hat)
        
        t += 1
        log_p = new_log_p
        
    if tempo :
        print('t =',t)
    
    #Estimation de Z
    Z = (Yc - Xc) @ W_hat @ nla.inv(np.transpose(W_hat)@W_hat)
    
    return W_hat,Z,Lambda_hat,sigma2

def MLE_Gauss(Y):
    '''Estimation de la moyenne et de la variance d'un N-échantillon Gaussien multivarié de covariance isotrope.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés, i.i.d. de loi normale.
        
    Renvois
    -------
    mu : 1-D ndarray,
        Estimation de la moyenne de la loi des vecteurs lignes de Y.
    
    sigma2 : float,
        Estimation de la variance de la loi des vecteurs lignes de Y.
    '''
    N,D = np.shape(Y)
    mu_ML = np.mean(Y,axis=0)
    sigma2_ML = np.var(Y)*(N/(N-1))
    return mu_ML,sigma2_ML

# Fonctions de clustering (CLF)

In [ ]:
'''=======================
    Fonctions de clustering
    =======================
    
    ================================== ===================================================================
    Contient les fonctions suivantes :
    ---------------------------------- -------------------------------------------------------------------
    HAC_SL                             Clustering Agglomératif Hiérarchique, en distance Single-Linkage.
    HAC_CL                             Clustering Agglomératif Hiérarchique, en distance Complete-Linkage.
    HAC_AL                             Clustering Agglomératif Hiérarchique, en distance Average-Linkage.
    HAC_Ward                           Clustering Agglomératif Hiérarchique, en distance de Ward.
    K_means                            Algorithme itératif K-means, ou "K-moyennes".
    K_means_FPC                        Algorithme itératif K-means ++.
    K_medoids                          Algorithme itératif K-medoids.
    Lapras                             Clustering Spectral.
    K_opt                              Estimation du nombre de clusters.
    ================================== ===================================================================
'''

In [13]:
def HAC_SL(X,K,tempo=False):
    '''Clustering Agglomératif Hiérarchique, en distance Single-Linkage.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    tempo : bool, optional,
        Ne sert à rien, quelle que soit sa valeur.
        Mis sur False par défaut.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    clusters = sklclu.AgglomerativeClustering(K,linkage='single').fit(X)
    omega = clusters.labels_
    
    return omega

def HAC_CL(X,K,tempo=False):
    '''Clustering Agglomératif Hiérarchique, en distance Complete-Linkage.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    tempo : bool, optional,
        Ne sert à rien, quelle que soit sa valeur.
        Mis sur False par défaut.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    clusters = sklclu.AgglomerativeClustering(K,linkage='complete').fit(X)
    omega = clusters.labels_
    
    return omega

def HAC_AL(X,K,tempo=False):
    '''Clustering Agglomératif Hiérarchique, en distance Average-Linkage.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    tempo : bool, optional,
        Ne sert à rien, quelle que soit sa valeur.
        Mis sur False par défaut.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    clusters = sklclu.AgglomerativeClustering(K,linkage='average').fit(X)
    omega = clusters.labels_
    
    return omega

def HAC_Ward(X,K,tempo=False):
    '''Clustering Agglomératif Hiérarchique, en distance de Ward.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    tempo : bool, optional,
        Ne sert à rien, quelle que soit sa valeur.
        Mis sur False par défaut.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    clusters = sklclu.AgglomerativeClustering(K,linkage='ward').fit(X)
    omega = clusters.labels_
    
    return omega

def K_means(X,omega,nb_steps=100,tempo=True):
    '''Algorithme itératif K-means, ou "K-moyennes".
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        L'algorithme s'arrête de lui-même si jamais la configuration obtenue est déjà stable.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme.
        Mis sur True par défaut.
        
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(X)
    occ = ufc.occurences(omega)
    K = len(occ)
    if np.any(occ==0) :
        print("K_means : Clusters vides")
    
    omega_hat = omega.copy()
    t = 0
    dist = 1
    M = np.zeros((K,D))
    
    while dist > 0 and t < nb_steps :
        
        #Recalcul des centres
        tri_X = ufc.tri(X,omega_hat,K)
        new_M = np.zeros((K,D))
        for k in range(K) :
            if len(tri_X[k]) > 0:
                new_M[k] = np.mean(tri_X[k],axis = 0)
            else :
                new_M[k] = M[k]
        M = new_M
    
        #Recalcul des clusters
        new_omega = np.zeros(N)
        for n in range(N) :
            x = X[n]
            dists_L2 = np.array([np.sum((x-M[k])**2) for k in range(K)])
            new_omega[n] = np.argmin(dists_L2)
        
        dist = np.sum((omega_hat-new_omega)**2)
        omega_hat = new_omega.astype(int)
        t += 1
    
    if tempo :
        print('t = ',t)
    
    return omega_hat

def K_means_FPC(X,K,determ=True,nb_steps=100,tempo=True):
    '''Algorithme K-means ++, i.e. K-means mais qui s'initialise en prenant comme centres des clusters des vecteurs du jeu de données X.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    determ : bool, optional,
        Si mis sur True, l'algorithme s'initialise en prenant comme centres des deux premiers clusters les deux vecteurs les plus éloignés, puis prend successivement, comme centres des autres clusters, les vecteurs dont la distance L2 minimale aux vecteurs déjà sélectionnés est maximale. 
        Si mis sur False, l'algorithme s'initialise en prenant comme centre du premier cluster un vecteur uniformément au hasard, puis prend successivement, comme centres des autres clusters, des vecteurs au hasard, chacun pondérés proportionnellement à leur distance L2 minimale aux vecteurs déjà sélectionnés.
        Mis sur True par défaut.
        
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        L'algorithme s'arrête de lui-même si jamais la configuration obtenue est déjà stable.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
        
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N = len(X)
    if N < K :
        print("N < K")
    else :
        
        #Initialisation
        
        if determ :
            Pdm = ufc.dissim_L2(X)
            ind_max = np.argmax(Pdm)
            n0 = int(ind_max/N)
            n1 = ind_max%N
            
            M = np.array([X[n0],X[n1]])
            
            for k in range(2,K):
            
                dists_CP = np.array([ufc.Dist_CP(x,M) for x in X])
                n_k = np.argmax(dists_CP)
                
                M_list = list(M)
                M_list.append(X[n_k])
                M = np.array(M_list)
        
        else :
            n0 = rd.choice(N)
            M = np.array([X[n0]])
            
            for k in range(1,K):
                
                dists_CP = np.array([ufc.Dist_CP(x,M) for x in X])
                
                tot_dist = np.sum(dists_CP)
                probas = dists_CP/tot_dist
                n_k = rd.choice(N,p=probas)
                
                M_list = list(M)
                M_list.append(X[n_k])
                M = np.array(M_list)
            
        omega = (-np.ones(N)).astype(int)
        for n in range(N):
            x = X[n]
            dists_L2 = np.array([np.sum((x-M[k])**2) for k in range(K)])
            omega[n] = int(np.argmin(dists_L2))
        
        #K-means
        return K_means(X,omega,nb_steps,tempo)

def K_medoids(X,omega,nb_steps=100,tempo=True):
    '''Algorithme itératif K-medoids, ou "K-médoïdes", ou encore "K-centroïdes" (ew).
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        L'algorithme s'arrête de lui-même si jamais la configuration obtenue est déjà stable.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme.
        Mis sur True par défaut.
        
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(X)
    occ = ufc.occurences(omega)
    K = len(occ)
    if np.any(occ==0) :
        print("K_means : Clusters vides")
    
    omega_hat = omega.copy()
    t = 0
    dist = 1
    M = np.zeros((K,D))
    
    while dist > 0 and t < nb_steps :
        
        #Recalcul des médoïdes
        tri_X = ufc.tri(X,omega_hat,K)
        for k in range(K):
            dist_sums = np.array([np.sum(np.array([np.sum((x-y)**2) for x in tri_X[k]])) for y in tri_X[k]])
            n_k = np.argmin(dist_sums)
            M[k] = X[n_k]
        
        #Recalcul des clusters
        new_omega = np.zeros(N)
        for n in range(N) :
            x = X[n]
            dists_L2 = np.array([np.sum((x-M[k])**2) for k in range(K)])
            new_omega[n] = np.argmin(dists_L2)
        
        dist = np.sum((omega_hat-new_omega)**2)
        omega_hat = new_omega.astype(int)
        t += 1
    
    if tempo :
        print('t = ',t)
    
    return omega_hat

#Vecteurs propres du Laplacien du graphe

def Lapras(X,K,determ=True,nb_steps=100,tempo=True):
    '''Clustering Spectral utilisant, pour matrice de similarité, la matrice de dissimilarité donnée par dissim_L2 renormalisée, dont chaque coefficient est passé par la fonction inverse de l'exponentielle.
        (La fonction s'appelle Lapras parce que c'est le nom en anglais d'un Pokémon dont le nom en japonais est Laplace.)
        
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K : int,
        Nombre de clusters à former.
    
    determ : bool, optional,
        Si mis sur True, l'algorithme K-means++ s'initialisera en prenant comme centres des deux premiers clusters les deux vecteurs les plus éloignés, puis prend successivement, comme centres des autres clusters, les vecteurs dont la distance L2 minimale aux vecteurs déjà sélectionnés est maximale. 
        Si mis sur False, l'algorithme K-means++ s'initialisera en prenant comme centre du premier cluster un vecteur uniformément au hasard, puis prend successivement, comme centres des autres clusters, des vecteurs au hasard, chacun pondérés proportionnellement à leur distance L2 minimale aux vecteurs déjà sélectionnés.
        Mis sur True par défaut.
        
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        L'algorithme s'arrête de lui-même si jamais la configuration obtenue est déjà stable.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
        
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(X)
    
    #Initialisation
    Pdm = ufc.dissim_L2(X)
    alpha = np.mean(Pdm)
    Psm = np.exp(-N/(N-1)/alpha*Pdm)
    Lokh = ufc.Lap(Psm)

    #EVP
    SpL,P = nla.eig(Lokh)
    P2 = mbu.normalize(P)
    ordre = np.sort(SpL)
    inlist = [k for k in range(N) if SpL[k] in ordre[:K]]
    tU = np.array([P2[:,k] for k in inlist])
    T = np.transpose(mbu.normalize(tU))
    omega = K_means_FPC(T,K,determ=determ,nb_steps=nb_steps,tempo=tempo)
    
    return omega

#K optimal
def K_opt(X,K_min=2,alpha=None,detail=False):
    '''Estimation du nombre de clusters à former à partir du spectre de la matrice de dissimilarité donnée par la fonction dissim_L2.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à clusteriser.
    
    K_min : int, optional,
        Nombre minimal de clusters à dégager.
    
    alpha : float, optional,
        Coefficient de renormalisation de la matrice de dissimilarité donnée par la fonction dissim_L2.
        Si mis sur None, prendra comme valeur l'inverse de la moyenne des distances L2 entre les vecteurs.
    
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le détail graphique de l'estimation du nombre de clusters.
        Mis sur False par défaut.
        
    Renvois
    -------
    K : int,
        Estimation du nombre optimal de clusters à partir du plus grand saut dans le log de l'opposé du spectre de la la matrice de dissimilarité donnée par la fonction dissim_L2, renormalisée.
    '''
    N,D = np.shape(X)    
    Pdm = ufc.dissim_L2(X)
    
    if type(alpha) == type(None):
        alpha = 1.0/np.mean(Pdm)
    
    norm_Pdm = alpha*Pdm
    SpP,Q = nla.eig(norm_Pdm)
    ordre = np.sort(SpP)[:-1]
    
    logm_ordre = np.log(-ordre)
    diff = logm_ordre[:-1]-logm_ordre[1:]
    
    if K_min >= 2:
        diff[:K_min-2] = [0]*(K_min-2)
    K_star = int(np.argmax(diff)) + 2
    
    if detail:
        
        plt.figure()
        plt.step(np.arange(2,N+1),logm_ordre)
        plt.plot([K_star,K_star],[logm_ordre[0],logm_ordre[-1]],'--',label='$K_{star}$')
        plt.title("Log de l'opposé du spectre de la matrice de dissimilarité")
        plt.legend()
        plt.figure()
    
    return K_star

# Fonctions pour reconstruire et représenter les données (DRR)

In [ ]:
'''======================================================
    Fonctions pour reconstruire ou représenter les données
    ======================================================
    
    ================================== ==================================================================================
    Contient les fonctions suivantes :
    ---------------------------------- ----------------------------------------------------------------------------------
    PCA_rec                            Reconstruction des vecteurs observés après (P)PCA
                                       (Adapté aux modèles (M.1) et (M.5.1)).
    RCA_rec                            Reconstruction des vecteurs observés après RCA
                                       (Adapté aux modèles (M.2) et (M.5.2)).
    MFA_rec_1                          Reconstruction des vecteurs observés après clustering
                                       puis (P)PCA sur les différents clusters (Adapté au modèle (M.4.1)).
    MFA_rec_2                          Reconstruction des vecteurs observés après clustering
                                       puis RCA sur les différents clusters (Adapté au modèle (M.4.2)).
    CA_graph                           Représentation graphique de la différence entre deux ensembles de vecteurs
                                       (Adapté aux modèles (M.1) et (M.2)).
    MFA_graph                          Représentation graphique de la différence entre deux ensembles de vecteurs,
                                       coloriés selon leur clusters (Adapté aux modèles (M.4) et (M.5)).
    discard                            Amputation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1.
    disarg                             Permutation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1
                                       (opération inverse de la fonction rearg).
    restit                             Ajout de colonnes vides à une matrice de vecteurs selon un vecteur de 0 et de 1.
    rearg                              Permutation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1
                                       (opération inverse de la fonction disarg).
    da_matrix                          Matrice de permutation associée à la permutation disarg(.,iota)
    ra_matrix                          Matrice de permutation associée à la permutation rearg(.,iota)
    FS_rec1                            Reconstruction des vecteurs observés après (P)PCA puis sélection de variables
                                       (Adapté aux modèles (M.6.1) et (M.6.3)).
    FS_rec2                            Reconstruction des vecteurs observés après RCA puis sélection de variables
                                       (Adapté aux modèles (M.6.2) et (M.6.4)).
    FS_mixrec1                         Reconstruction des vecteurs observés après clustering, (P)PCA cluster par cluster,
                                       puis sélection de variables cluster par cluster (Adapté au modèle (M.6.5)).
    FS_mixrec2                         Reconstruction des vecteurs observés après clustering, RCA cluster par cluster,
                                       puis sélection de variables cluster par cluster (Adapté au modèle (M.6.6)).
    FS_sperec1                         Reconstruction des vecteurs observés après clustering, sélection de variables,
                                       puis (P)PCA (Adapté au modèle (M.6.7)).
    FS_sperec2                         Reconstruction des vecteurs observés après clustering, sélection de variables,
                                       puis RCA (Adapté au modèle (M.6.8)).
    ================================== ==================================================================================
'''

In [11]:
def PCA_rec(W,Z,mu):
    '''Reconstruction des vecteurs observés après (P)PCA.
    (Adapté aux modèles (M.1) et (M.5.1))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    mu : 1-D ndarray,
        Vecteur de taille D, supposé être la moyenne des observations.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    Y = Z @ np.transpose(W) + mu
    return Y

def RCA_rec(W,Z,V,X,mu):
    '''Reconstruction des vecteurs observés après RCA.
    (Adapté aux modèles (M.2) et (M.5.2))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables.
        L'algorithme s'assure que ces vecteurs sont centrés en les recentrant.
    
    mu : 1-D ndarray,
        Vecteur de taille D, supposé être la moyenne des observations.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    Xc = X - np.mean(X,axis=0)
    Y = Z @ np.transpose(W) + Xc @ np.transpose(V) + mu
    return Y

def MFA_rec_1(thetas,Z,omega):
    '''Reconstruction des vecteurs observés après clustering puis (P)PCA sur les différents clusters.
    (Adapté au modèle (M.4.1))
    
    Paramètres
    ----------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    D,L = np.shape(thetas[0][0])
    N = len(omega)
    Y = np.zeros((N,D))
    
    for n in range(N):
        W,mu,sigma2 = thetas[omega[n]]
        z = Z[n]
        Y[n] = W@z + mu
    
    return Y

def MFA_rec_2(thetas,Z,X,omega):
    '''Reconstruction des vecteurs observés après clustering puis RCA sur les différents clusters.
    (Adapté au modèle (M.4.2))
    
    Paramètres
    ----------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,V,mu,sigma2] où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster.
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables.
        L'algorithme s'assure que ces vecteurs sont centrés en les recentrant, cluster par cluster.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier indiquant le numéro du cluster auquel appartient le n-ième individu.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    D,L = np.shape(thetas[0][0])
    N = len(omega)
    Y = np.zeros((N,D))
    tri_X = ufc.tri(X,omega)
    
    for n in range(N):
        W,V,mu,sigma2 = thetas[omega[n]]
        z = Z[n]
        x = X[n] - np.mean(tri_X[omega[n]],axis=0)
        
        Y[n] = W@z + V@x + mu
    
    return Y

def CA_graph(Y,Y_hat):
    '''Représentation graphique de la différence entre deux ensembles de même nombre de vecteurs de même taille.
    (Adapté aux modèles (M.1) et (M.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à comparer.
    
    Y_hat : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à comparer.
    
    Renvois
    -------
    None
    '''
    N,D = np.shape(Y)
    
    for j in range(int(D/2)):
        
        plt.figure()
        
        plt.scatter(Y[:,2*j],Y[:,2*j+1],label='$Y$')
        plt.scatter(Y_hat[:,2*j],Y_hat[:,2*j+1],label='$\hat{Y}$')
        
        for n in range(N):
            plt.plot([Y[n][2*j],Y_hat[n][2*j]],[Y[n][2*j+1],Y_hat[n][2*j+1]],color='black')
            
        plt.legend()
        plt.show()

def MFA_graph(Y,Y_hat,omega_hat,omega=None,labels=None):
    '''Représentation graphique de la différence entre deux ensembles de même nombre de vecteurs de même taille, coloriés selon leur clusters.
    (Adapté aux modèles (M.4) et (M.5))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à comparer.
    
    Y_hat : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à comparer.
    
    omega_hat : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu de l'ensemble de vecteurs Y_hat.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu de l'ensemble de vecteurs Y.
        Si mis sur None, prend la valeur de omega_hat.
        Sinon, sera permuté pour être optimal avec omega_hat avec la fonction perm_opt avant représentation graphique.
        Mis sur None par défaut.
        
    labels : list of str, optional,
        Liste de K éléments, contenant les noms des clusters.
    
    Renvois
    -------
    None
    '''
    N,D = np.shape(Y)
    K = int(max(omega_hat) + 1)
    
    if type(omega) == type(None):
        s_omega = omega_hat
        K = int(max(omega_hat) + 1)
    else:
        s_opt = ufc.perm_opt(omega,omega_hat)
        s_omega = np.array([s_opt[k] for k in omega]).astype(int)
        K = int(max(omega_hat) + 1)
        
    tri_Y = ufc.tri(Y,s_omega,K)
    tri_Y_hat = ufc.tri(Y_hat,omega_hat,K)
    colors_orig = ['#802020','#E08080','#208020','#80E080','#202080','#8080E0','#808020','#E0E080','#802080','#E080E0','#208080','#80E0E0','#805020','#E0B080','#508020','#B0E080','#208050','#80E0B0','#205080','#80B0E0','#502080','#B080E0','#802050','#E080B0','#202020','#E0E0E0']
    nb_cyc = int(np.ceil(K/len(colors_orig)))
    colors = colors_orig*nb_cyc
    
    if type(labels) == type(None):
        for j in range(int(D/2)):
            
            plt.figure()
            
            for k in range(K):
                if len(tri_Y[k]) > 0:
                    plt.scatter(tri_Y[k][:,2*j],tri_Y[k][:,2*j+1],label='$Y$',color=colors[2*k])
                if len(tri_Y_hat[k]) > 0:
                    plt.scatter(tri_Y_hat[k][:,2*j],tri_Y_hat[k][:,2*j+1],label='$\hat{Y}$',color=colors[2*k+1])
            
            for n in range(N):
                plt.plot([Y[n][2*j],Y_hat[n][2*j]],[Y[n][2*j+1],Y_hat[n][2*j+1]],color='black')
            
            plt.legend()
            plt.show()
    else :
        for j in range(int(D/2)):
            
            plt.figure()
            
            for k in range(K):
                if len(tri_Y[k]) > 0:
                    plt.scatter(tri_Y[k][:,2*j],tri_Y[k][:,2*j+1],label='$Y$ - '+labels[k],color=colors[2*k])
                if len(tri_Y_hat[k]) > 0:
                    plt.scatter(tri_Y_hat[k][:,2*j],tri_Y_hat[k][:,2*j+1],label='$\hat{Y} - $'+labels[k],color=colors[2*k+1])
            
            for n in range(N):
                plt.plot([Y[n][2*j],Y_hat[n][2*j]],[Y[n][2*j+1],Y_hat[n][2*j+1]],color='red')
            
            plt.legend()
            plt.show()

def discard(Y,iota):
    '''Amputation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs à amputer.
    
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, où 0 signifie que la colonne est à jeter, et 1 signifie que la colonne est à garder.
        On note U le nombre de 1 dans iota.
        
    Renvois
    -------
    Y_tilde : 2-D ndarray,
        Matrice de taille (N,U) dont les colonnes ont été amputées selon iota.
    '''
    N,D = np.shape(Y)
    D1 = len(iota)
    
    if D != D1:
        print("Le nombre de dimensions ne correspond pas")
    else :
        if np.any((iota-iota**2).astype(bool)):
            print("Vecteur non-booléen")
        else :
            Y_tilde = np.transpose(np.array([Y[:,d] for d in range(D) if iota[d]]))
            return Y_tilde

def disarg(Y,iota):
    '''Permutation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1, opération inverse de la fonction rearg.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les colonnes sont à permuter.
    
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, où 0 signifie que la colonne est à placer à la fin, et 1 signifie que la colonne est à placer au début.
        
    Renvois
    -------
    Y_tilde : 2-D ndarray,
        Matrice de taille (N,D) dont les colonnes ont été permutées selon iota.
    '''
    Y_v = discard(Y,iota)
    Y_u = discard(Y,1-iota)
    Y_tilde = np.concatenate([Y_v,Y_u],axis=1)
    return Y_tilde
        
def restit(Y,iota):
    '''Ajout de colonnes vides à une matrice de vecteurs selon un vecteur de 0 et de 1.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,U) dont les lignes sont les vecteurs à amputer.
    
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, dont le nombre de 1 est U, où 0 signifie qu'une colonne remplie de 0 est à placer, et 1 signifie qu'une colonne de Y est à placer.
        
    Renvois
    -------
    Y_tilde : 2-D ndarray,
        Matrice de taille (N,D) dont les colonnes ont été aérées de 0 selon iota.
    '''
    N,U = np.shape(Y)
    D = len(iota)
    
    if np.any((iota-iota**2).astype(bool)):
        print("Vecteur non-booléen")
    else :
        iota_inv = np.array([d for d in range(D) if iota[d]])
        if len(iota_inv) != U :
            print("Le nombre de dimensions ne correspond pas")
        else :
            Y_tilde = np.zeros((N,D))
            for u in range(U):
                Y_tilde[:,iota_inv[u]] = Y[:,u]
            return Y_tilde

def rearg(Y,iota):
    '''Permutation de colonnes d'une matrice de vecteurs selon un vecteur de 0 et de 1, opération inverse de la fonction disarg.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les colonnes sont à permuter.
    
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, dont le nombre de 1 est U, où 0 signifie que la colonne est à prendre parmi les D-U dernières, et 1 signifie que la colonne est à prendre parmi les U premières.
        
    Renvois
    -------
    Y_tilde : 2-D ndarray,
        Matrice de taille (N,D) dont les colonnes ont été permutées selon iota.
    '''
    N,D = np.shape(Y)
    D1 = len(iota)
    
    if D != D1:
        print("Le nombre de dimensions ne correspond pas")
    else :
        if np.any((iota-iota**2).astype(bool)):
            print("Vecteur non-booléen")
        else :
            Dv = np.sum(iota)
            Y_v = restit(Y[:,:Dv],iota)
            Y_u = restit(Y[:,Dv:],1-iota)
        
    return Y_u + Y_v

def da_matrix(iota):
    '''Matrice de permutation associée à la permutation disarg(.,iota), inverse de la matrice ra_matrix(iota).
    
    Paramètres
    ----------
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, où 0 signifie que la colonne est à placer à la fin, et 1 signifie que la colonne est à placer au début.
        
    Renvois
    -------
    R : 2-D ndarray,
        Matrice de permutation carrée d'ordre D associée à la permutation disarg(.,iota).
    '''
    D = len(iota)
    if np.any((iota-iota**2).astype(bool)):
        print("Vecteur non-booléen")
    else:
        R = np.zeros((D,D)).astype(int)
        Dv = np.sum(iota)
        
        u = 0
        v = 0
        for d in range(D):
            if iota[d]:
                R[v][d] = 1
                v += 1
            else :
                R[Dv+u][d] = 1
                u += 1
        return R

def ra_matrix(iota):
    '''Matrice de permutation associée à la permutation rearg(.,iota), inverse de la matrice da_matrix(iota).
    
    Paramètres
    ----------
    iota : 1-D ndarray,
        Vecteur de taille D rempli de 0 et de 1, dont le nombre de 1 est U, où 0 signifie que la colonne est à prendre parmi les D-U dernières, et 1 signifie que la colonne est à prendre parmi les U premières.
        
    Renvois
    -------
    R : 2-D ndarray,
        Matrice de permutation carrée d'ordre D associée à la permutation rearg(.,iota).
    '''
    D = len(iota)
    if np.any((iota-iota**2).astype(bool)):
        print("Vecteur non-booléen")
    else:
        R = np.zeros((D,D)).astype(int)
        Dv = np.sum(iota)
        
        u = 0
        v = 0
        for d in range(D):
            if iota[d]:
                R[d][v] = 1
                v += 1
            else :
                R[d][Dv+u] = 1
                u += 1
        return R

def FS_rec1(W,Z,mu,iota):
    '''Reconstruction des vecteurs observés après (P)PCA puis sélection de variables.
    (Adapté aux modèles (M.6.1) et (M.6.3))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    mu : 1-D ndarray,
        Vecteur de taille D, supposé être la moyenne des observations.
    
    iota : 1-D ndarray,
        Vecteur de taille D, rempli de 0 et de 1, dont le nombre de 1 est U, où 0 signifie qu'une colonne remplie de 0 est à placer, et 1 signifie qu'une colonne du produit de Z par la transposée de W est à placer.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    Y_tilde = Z@np.transpose(W)
    Y_hat = restit(Y_tilde, iota) + mu
    
    return Y_hat

def FS_rec2(W,Z,V,X,mu,iota):
    '''Reconstruction des vecteurs observés après RCA puis sélection de variables.
    (Adapté aux modèles (M.6.2) et (M.6.4))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables.
        L'algorithme s'assure que ces vecteurs sont centrés en les recentrant.
    
    mu : 1-D ndarray,
        Vecteur de taille D, supposé être la moyenne des observations.
    
    iota : 1-D ndarray,
        Vecteur de taille D, rempli de 0 et de 1, dont le nombre de 1 est U, où 0 signifie qu'une colonne remplie de 0 est à placer, et 1 signifie qu'une colonne du produit de Z par la transposée de W est à placer.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    mu_X = np.mean(X,axis=0)
    Xc = X - mu_X
    
    Y_tilde = Z@np.transpose(W)
    Y_hat = restit(Y_tilde, iota) + Xc@np.transpose(V) + mu
    
    return Y_hat

def FS_mixrec1(thetas,Z,omega,iotas):
    '''Reconstruction des vecteurs observés après clustering, (P)PCA cluster par cluster, puis sélection de variables cluster par cluster.
    (Adapté au modèle (M.6.5))
    
    Paramètres
    ----------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) remplie de 0 et de 1, dont chaque ligne contient U fois le nombre 1, où, pour chaque ligne, 0 signifie que, pour le cluster correspondant, une colonne remplie de 0 est à placer, et 1 signifie que, pour le cluster correspondant, une colonne du produit de Z par la transposée du W correspondant est à placer.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    N = len(omega)
    K,D = np.shape(iotas)
    Y_hat = np.zeros((N,D))
    
    for n in range(N):
        
        k = omega[n]
        W,mu,sigma2 = thetas[k]
        Y_tilde_n = np.array([W@Z[n]])
        Y_hat[n] = (restit(Y_tilde_n,iotas[k]))[0] + mu
    
    return Y_hat

def FS_mixrec2(thetas,Z,X,omega,iotas):
    '''Reconstruction des vecteurs observés après clustering, RCA cluster par cluster, puis sélection de variables cluster par cluster.
    (Adapté au modèle (M.6.6))
    
    Paramètres
    ----------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,V,mu,sigma2] où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster.
            - V est une matrice de taille (D,C) d'effets fixes du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables.
        L'algorithme s'assure que ces vecteurs sont centrés en les recentrant, cluster par cluster.
        
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) remplie de 0 et de 1, dont chaque ligne contient U fois le nombre 1, où, pour chaque ligne, 0 signifie que, pour le cluster correspondant, une colonne remplie de 0 est à placer, et 1 signifie que, pour le cluster correspondant, une colonne du produit de Z par la transposée du W correspondant est à placer.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs reconstruits.
    '''
    N = len(omega)
    K,D = np.shape(iotas)
    Y_hat = np.zeros((N,D))
    
    tri_X = ufc.tri(X,omega)
    
    for n in range(N):
        
        k = omega[n]
        W,V,mu,sigma2 = thetas[k]
        x = X[n] - np.mean(tri_X[k],axis=0)
        Y_tilde_n = np.array([W@Z[n]])
        Y_hat[n] = (restit(Y_tilde_n,iotas[k]))[0] + V@x + mu
    
    return Y_hat

def FS_sperec1(eta,Zv,Zu,iota):
    '''Reconstruction des vecteurs observés après clustering, sélection de variables, puis (P)PCA.
    (Adapté au modèle (M.6.7))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,mu,nu,sigma2,tau2] où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux sur lesquels la loi des variables aléatoires change en fonction du cluster.
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux sur lesquels la loi des variables aléatoires ne change pas en fonction du cluster.
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas en fonction du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas en fonction du cluster.
    
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents dont la loi change en fonction du cluster.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents dont la loi ne change pas en fonction du cluster.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables.
        L'algorithme s'assure que ces vecteurs sont centrés en les recentrant, cluster par cluster.
    
    iota : 1-D ndarray,
        Vecteur de taille Dv+Du rempli de 0 et de 1, dont le nombre de 1 est Dv, où 0 signifie que la colonne est à prendre parmi celles du produit de Zu avec la transposée de Wu, et 1 signifie que la colonne est à prendre parmi celles du produit de Zv avec la transposée de Wv.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,Dv+Du) dont les lignes sont les vecteurs reconstruits.
    '''
    Wv,Wu,mu,nu,sigma2,tau2 = eta
    
    Dv,Lv = np.shape(Wv)
    Du,Lu = np.shape(Wu)
    N1,Lv1 = np.shape(Zv)
    N2,Lu1 = np.shape(Zu)
    D = len(mu)
    
    if N1 != N2 :
        print("Erreur de dimensions sur Zv et Zu")
    if D != Du+Dv:
        print("Erreur de dimensions sur Wu, Wv et mu")
    if Lv != Lv1 :
        print("Erreur de dimensions sur Wv et Zv")
    if Lu != Lu1 :
        print("Erreur de dimensions sur Wu et Zu")
        
    Y_tilde = np.concatenate([Zv@np.transpose(Wv),Zu@np.transpose(Wu)],axis=1)
    Y_hat = rearg(Y_tilde,iota) + mu
    
    return Y_hat
        
def FS_sperec2(eta,Zv,Zu,X,iota):
    '''Reconstruction des vecteurs observés après clustering, sélection de variables, puis (P)PCA.
    (Adapté au modèle (M.6.8))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,V,mu,nu,sigma2,tau2] où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux sur lesquels la loi des variables aléatoires change en fonction du cluster.
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux sur lesquels la loi des variables aléatoires ne change pas en fonction du cluster.
            - V est une matrice de taille (Dv+Du,C) d'effets fixes.
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas en fonction du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas en fonction du cluster.
    
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents dont la loi change en fonction du cluster.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents dont la loi ne change pas en fonction du cluster.
    
    iota : 1-D ndarray,
        Vecteur de taille Dv+Du rempli de 0 et de 1, dont le nombre de 1 est Dv, où 0 signifie que la colonne est à prendre parmi les Du dernières, et 1 signifie que la colonne est à prendre parmi les Dv premières.
        
    Renvois
    -------
    Y : 2-D ndarray,
        Matrice de taille (N,Dv+Du) dont les lignes sont les vecteurs reconstruits.
    '''
    Wv,Wu,V,mu,nu,sigma2,tau2 = eta
    
    Dv,Lv = np.shape(Wv)
    Du,Lu = np.shape(Wu)
    N1,Lv1 = np.shape(Zv)
    N2,Lu1 = np.shape(Zu)
    N,C = np.shape(X)
    D1,C1 = np.shape(V)
    D = len(mu)
    
    if N1 != N2 :
        print("Erreur de dimensions sur Zv et Zu")
    if D != Du+Dv:
        print("Erreur de dimensions sur Wu, Wv et mu")
    if Lv != Lv1 :
        print("Erreur de dimensions sur Wv et Zv")
    if Lu != Lu1 :
        print("Erreur de dimensions sur Wu et Zu")
    if D != D1 :
        print("Erreur de dimensions sur V et mu")
    if N != N1 :
        print("Erreur de dimensions sur Zv et X")
    if N != N2 :
        print("Erreur de dimensions sur Zu et X")
    if C != C1 :
        print("Erreur de dimensions sur V et X")
        
    Xc = X - np.mean(X,axis=0)
    Y_tilde = np.concatenate([Zv@np.transpose(Wv),Zu@np.transpose(Wu)],axis=1)
    Y_hat = rearg(Y_tilde,iota) + Xc@np.transpose(V) + mu
    
    return Y_hat

# Fonctions utiles pour la sélection de variables (UFS)

In [ ]:
'''===============================================
    Fonctions utiles pour la sélection de variables
    ===============================================
    
    ================================== ==========================================================================
    Contient les fonctions suivantes :
    ---------------------------------- --------------------------------------------------------------------------
    cor_emp                            Matrice de corrélation empirique des matrices de vecteurs
    FWIR                               "Feature-Wise Inertia Rate" ou "Part d'inertie, variable par variable".
    U_opt                              Nombre optimal de dimensions à conserver, estimé par la "méthode du saut".
    iotate                             Estimation du vecteur induisant une sélection de variables
                                       (comparaison des parts d'inerties, variable par variable).
    iotate_2                           Estimation du vecteur induisant une sélection de variables
                                       (comparaison des coefficients de corrélation).
    ================================== ==========================================================================
'''

In [6]:
def cor_emp(Y,Z):
    '''Matrice de corrélation empirique des matrices de vecteurs Y et Z.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D1) dont les lignes sont des réalisations supposément indépendantes de vecteurs aléatoires de taille D.
    
    Z : 2-D ndarray,
        Matrice de taille (N,D2) dont les lignes sont des réalisations supposément indépendantes de vecteurs aléatoires de taille L.
        
    Renvois
    -------
    Cor : 2-D ndarray,
        Matrice de taille (D2,D1) dont le coefficient pour chaque ligne et chaque colonne est le coefficient de corrélation entre la ligne de Z et la colonne de Y correspondantes.
    '''
    N1,D = np.shape(Y)
    N2,L = np.shape(Z)
    
    if N1 != N2:
        print("Y et Z sont de tailles différentes")
    else :
        N = N1
        
        mu_Y = np.mean(Y,axis=0)
        vars_Y = np.var(Y,axis=0)
        Yc = Y - mu_Y
        
        mu_Z = np.mean(Z,axis=0)
        vars_Z = np.var(Z,axis=0)
        Zc = Z - mu_Z
        
        Cor = 1/N *(np.diag(1/np.sqrt(vars_Z)) @ np.transpose(Zc) @ Yc @ np.diag(1/np.sqrt(vars_Y)))
        
        return Cor

def FWIR(X,omega):
    '''"Feature-Wise Inertia Rate" ou "Part d'inertie, variable par variable".
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs appartenant à différents clusters.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
       
    Renvois
    -------
    I_X : 2-D ndarray,
        Matrice de taille (N,D) dont le coefficient est la part d'inertie de l'individu en ligne selon l'axe en colonne.
    '''
    N,D = np.shape(X)
    N1 = len(omega)
    
    if N != N1:
        print("X et omega n'ont pas le même nombre d'individus")
    else :
        occ = ufc.occurences(omega)
        K = len(occ)
        if not np.all(occ):
            print("Au moins un des clusters est vide")
        else:

            tri_X = ufc.tri(X,omega)
            M = np.array([np.mean(tri_X[k],axis=0) for k in range(K)])
            mu_glob = np.mean(X,axis=0)

            I_loc = np.array([[(X[n][d] - M[omega[n]][d])**2 for d in range(D)] for n in range(N)])
            I_glob = np.array([[(X[n][d] - mu_glob[d])**2 for d in range(D)] for n in range(N)])
            I_X = I_glob/I_loc

            return I_X

def U_opt(contrib,U_min=1,detail=False):
    '''Nombre optimal de dimensions à conserver, estimé par la "méthode du saut".
    
    Paramètres
    ----------
    contrib : 1-D ndarray,
        Vecteur de taille D dont les coefficients sont d'autant plus grands que la dimension correspondante est utile pour le clustering.
    
    U_min : int, optional,
        Nombre minimal de variables à conserver.
        Mis sur 1 par défaut.
        
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le détail graphique de l'estimation du nombre de dimensions utiles.
        Mis sur False par défaut.
    
    Renvois
    -------
    U : int,
        Nombre optimal de dimensions à conserver.
    '''
    D = len(contrib)
    
    if U_min < 1:
        U_min = 1
    
    if D <= U_min:
        print("D inférieur ou égal à U_min")
    else:
        ordre = np.sort(contrib)
        diff_ordre = np.concatenate([[0],ordre[1:D-U_min+1] - ordre[:D-U_min]])
        U_star = D - int(np.argmax(diff_ordre))
        
        if detail:
            
            plt.figure()
            plt.step(np.arange(0,D),ordre,label='Contribution')
            plt.plot([D-U_star-1,D-U_star-1],[ordre[D-U_star-1],ordre[D-U_star]])
            plt.plot([D-U_star-1,D-U_star-1],[ordre[D-U_star-1],0],'--',label='$U_{star}$')
            plt.legend()
            plt.show()
        
        return U_star

def iotate(X,omega,U=None,detail=False):
    '''Estimation du vecteur iota induisant une sélection de variables, en comparant les parts d'inerties, variable par variable.
    
    Paramètres
    ----------
    X : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs appartenant à différents clusters.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    U : int, optional,
        Nombre de variables à conserver.
        Si mis sur None, le nombre de variables à conserver est estimé à partir de la fonction U_opt.
        Mis sur None par défaut.
        
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le détail graphique de la sélection de variables.
        Mis sur False par défaut.
    
    Renvois
    -------
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est à conserver, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N,D = np.shape(X)
    N1 = len(omega)
    
    if N != N1:
        print("X et omega n'ont pas le même nombre d'individus")
    else :
        occ = ufc.occurences(omega)
        K = len(occ)
        if not np.all(occ):
            print("Au moins un des clusters est vide")
        else:
            I_X = FWIR(X,omega)
            contrib = np.sum(I_X,axis=0)
            ordre = np.sort(contrib)

            if type(U) == type(None):
                U = U_opt(contrib,detail=detail)

            iota = np.array([int(contrib[d] in ordre[D-U:]) for d in range(D)])

            return iota

def iotate_2(Y,Z,U=None,dist=2,detail=False):
    '''Estimation du vecteur iota induisant une sélection de variables, en comparant les coefficients de corrélation, variable par variable.
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont des vecteurs latents dont la loi change selon les clusters.
    
    U : int, optional,
        Nombre de variables à conserver.
        Si mis sur None, le nombre de variables à conserver est estimé à partir de la fonction U_opt.
        Mis sur None par défaut.
    
    dist = int, float, or str, optional,
        Norme utilisée pour mesurer les colonnes de la matrice de corrélation empirique.
    
    detail : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le détail graphique de la sélection de variables.
        Mis sur False par défaut.
    
    Renvois
    -------
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est à conserver, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N1,D = np.shape(Y)
    N2,L = np.shape(Z)
    
    if N1 != N2:
        print("Y et Z sont de tailles différentes")
    else :
        N = N1
    
        Cor = cor_emp(Y,Z)
        
        if (type(dist) == int or type(dist) == float) and dist >= 1:
            contrib = (np.sum(np.abs(Cor)**dist,axis=0))**(1/dist)
        else :
            if dist == 'inf':
                contrib = np.max(np.abs(Cor),axis=0)
            else :
                print("Distance non-reconnue.")
            
            
        ordre = np.sort(contrib)
        
        if type(U) == type(None):
            U = U_opt(ordre,L,detail)
        
        brink = ordre[D-U]
        iota_hat = (contrib>=brink).astype(int)
        
        return iota_hat

# Fonctions d'estimation, mixtures de modèles (MFA)

In [ ]:
'''===============================================
    Fonctions d'estimation pour mixtures de modèles
    ===============================================
    
    ================================== ==================================================================================
    Contient les fonctions suivantes :
    ---------------------------------- ----------------------------------------------------------------------------------
    obs1                               Clustering, puis PPCA appliquée sur chaque cluster (Adapté pour le modèle (M.4.1))
    obs2                               Clustering, puis RCA appliquée sur chaque cluster (Adapté pour le modèle (M.4.2))
    lat1                               PPCA, puis clustering (Adapté pour le modèle (M.5.1)).
    lat2                               RCA, puis clustering (Adapté pour le modèle (M.5.2)).
    ================================== ==================================================================================
'''

In [13]:
def obs1(Y,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,tempo=True):
    '''Clustering, puis PPCA appliquée sur chaque cluster.
    (Adapté pour le modèle (M.4.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    L : int,
        Nombre de dimensions de l'espace latent.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    Renvois
    -------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA, cluster par cluster.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(Y)
    
    if type(omega) == type(None):
        
        if type(K) == type(None):
            K = clf.K_opt(Y)
                
        omega_hat = fun(Y,K,tempo=tempo)
        omega_hat = clf.K_means(Y,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    reclus = [[n for n in range(N) if omega_hat[n] == k] for k in range(K)]
    tri_Y = ufc.tri(Y,omega_hat)
    thetas_hat = [[] for k in range(K)]
    Z_hat = np.zeros((N,L))
    
    for k in range(K):
        
        y = tri_Y[k]
        card_k = len(y)
        mu_k = np.mean(y,axis=0)
        W_k_hat, Z_k_hat, sigma2_k_hat = sfa.PPCA(y,L)
        
        thetas_hat[k] = [W_k_hat, mu_k, sigma2_k_hat]
        for j in range(card_k):
            Z_hat[reclus[k][j]] = Z_k_hat[j]
        
    return thetas_hat, Z_hat, omega_hat

def obs2(Y,X,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,err=0.0,tempo=True):
    '''Clustering, puis RCA appliquée sur chaque cluster.
    (Adapté pour le modèle (M.4.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force, cluster par cluster.
    
    L : int,
        Nombre de dimensions latentes souhaité.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    Renvois
    -------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,V,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - V est la matrice de taille (D,C) d'effets fixes du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la RCA, cluster par cluster.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(Y)
    
    if type(omega) == type(None):
        
        if type(K) == type(None):
            K = clf.K_opt(Y)
                
        omega_hat = fun(Y,K,tempo=tempo)
        omega_hat = clf.K_means(Y,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    reclus = [[n for n in range(N) if omega_hat[n] == k] for k in range(K)]
    tri_Y = ufc.tri(Y,omega_hat)
    tri_X = ufc.tri(X,omega_hat)
    thetas_hat = [[] for k in range(K)]
    Z_hat = np.zeros((N,L))
    
    for k in range(K):
        
        y = tri_Y[k]
        x = tri_X[k]
        card_k = len(y)
        mu_k = np.mean(y,axis=0)
        W_k_hat, Z_k_hat, V_k_hat, sigma2_k_hat = sfa.ML_RCA(y,x,L,nb_steps=nb_steps,err=err,tempo=tempo)
        
        thetas_hat[k] = [W_k_hat, V_k_hat, mu_k, sigma2_k_hat]
        for j in range(card_k):
            Z_hat[reclus[k][j]] = Z_k_hat[j]
        
    return thetas_hat, Z_hat, omega_hat

def lat1(Y,L=None,K=None,omega=None,fun=clf.Lapras,nb_steps=100,tempo=True,latent=True):
    '''PPCA, puis clustering.
    (Adapté pour le modèle (M.5.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    latent : bool, optional,
        Si mis sur False, le clustering se fait sur la vecteurs observés.
        Si mis sur True, le clustering se fait sur la vecteurs latents.
        Mis sur True par défaut.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D, supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(Y)
    W_prov, Z_prov, sigma2_hat = sfa.PPCA(Y,L)
    D,L = np.shape(W_prov)
    
    D_W = np.diag(np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    D_W_inv = np.diag(1/np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    Z_hat = Z_prov @ D_W
    W_hat = W_prov @ D_W_inv
    
    mu_hat = np.mean(Y,axis=0)
    eta_hat = W_hat,mu_hat,sigma2_hat
    
    if type(omega) == type(None):
        
        if latent:
            if type(K) == type(None):
                K = clf.K_opt(Z_hat)

            omega_hat = fun(Z_hat,K,tempo=tempo)
            omega_hat = clf.K_means(Z_hat,omega_hat,nb_steps,tempo=tempo)
        else :
            if type(K) == type(None):
                K = clf.K_opt(Y)

            omega_hat = fun(Y,K,tempo=tempo)
            omega_hat = clf.K_means(Y,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    tri_Z = ufc.tri(Z_hat,omega_hat)
    thetas_hat = []
    
    for k in range(K):
        z = tri_Z[k]
        thetas_k = sfa.MLE_Gauss(z)
        thetas_hat.append(thetas_k)
        
    return eta_hat, thetas_hat, Z_hat, omega_hat

def lat2(Y,X,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,err=0.0,tempo=True):
    '''RCA, puis clustering.
    (Adapté pour le modèle (M.5.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int,
        Nombre de dimensions latentes souhaité.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
    
    latent : bool, optional,
        Si mis sur False, le clustering se fait sur la vecteurs observés.
        Si mis sur True, le clustering se fait sur la vecteurs latents.
        Mis sur True par défaut.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D, supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    N,D = np.shape(Y)
    N1,C = np.shape(X)
    W_prov, Z_prov, V_hat, sigma2_hat = sfa.ML_RCA(Y,X,L,nb_steps=nb_steps,err=err,tempo=tempo)
    
    D_W = np.diag(np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    D_W_inv = np.diag(1/np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    Z_hat = Z_prov @ D_W
    W_hat = W_prov @ D_W_inv
    
    mu_hat = np.mean(Y,axis=0)
    eta_hat = W_hat,V_hat,mu_hat,sigma2_hat
    
    if type(omega) == type(None):
        
        if type(K) == type(None):
            K = clf.K_opt(Z_hat)
                
        omega_hat = fun(Z_hat,K,tempo=tempo)
        omega_hat = clf.K_means(Z_hat,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    tri_Z = ufc.tri(Z_hat,omega_hat)
    thetas_hat = []
    
    for k in range(K):
        z = tri_Z[k]
        thetas_k = sfa.MLE_Gauss(z)
        thetas_hat.append(thetas_k)
        
    return eta_hat, thetas_hat, Z_hat, omega_hat

NameError: name 'clf' is not defined

# Fonctions pour simuler les données (SIM)

In [ ]:
'''====================================================
    Fonctions pour simuler des paramètres et des données
    ====================================================
    
    ================================== ======================================================================================
    Contient les fonctions suivantes :
    ---------------------------------- --------------------------------------------------------------------------------------
    param_lin                          Ensemble de paramètres pour le modèle linéaire Gaussien simple (Modèle (M.1)).
    param_cov                          Ensemble de paramètres pour le modèle linéaire Gaussien mixte (Modèle (M.2)).
    param_obsmix_1                     Ensembles de paramètres pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens simples (Modèle (M.4.1)).
    param_obsmix_2                     Ensembles de paramètres pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens mixtes (Modèle (M.4.2)).
    param_latmix_1                     Ensembles de paramètres pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens simples (Modèle (M.5.1)).
    param_latmix_2                     Ensembles de paramètres pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens mixtes (Modèle (M.5.2)).
    noisy_param_1                      Ensemble de paramètres pour le modèle linéaire Gaussien simple
                                       avec variables impertinentes (Modèle (M.6.1)).
    noisy_param_2                      Ensemble de paramètres pour le modèle linéaire Gaussien mixte
                                       avec variables impertinentes (Modèle (M.6.2)).
    noisy_param_latmix_1               Ensembles de paramètres pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens simples, avec variables impertinentes (Modèle (M.6.3)).
    noisy_param_latmix_2               Ensembles de paramètres pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens mixtes, avec variables impertinentes (Modèle (M.6.4)).
    noisy_param_obsmix_1               Ensembles de paramètres pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens simples, avec variables impertinentes (Modèle (M.6.5)).
    noisy_param_obsmix_2               Ensembles de paramètres pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens mixtes, avec variables impertinentes (Modèle (M.6.6)).
    noisy_param_spemix_1               Ensembles de paramètres pour la mixture sur seulement certaines dimensions des
                                       espaces observé et latent de modèles linéaires Gaussiens simples (Modèle (M.6.7)).
    noisy_param_spemix_2               Ensembles de paramètres pour la mixture sur seulement certaines dimensions des
                                       espaces observé et latent de modèles linéaires Gaussiens mixtes (Modèle (M.6.8)).
    
    sim_omega                          Vecteur aléatoire de taille rempli d'entiers positifs.
    
    data_lin                           Ensemble de données simulées pour le modèle linéaire Gaussien simple (Modèle (M.1)).
    data_cov                           Ensemble de données simulées pour le modèle linéaire Gaussien mixte (Modèle (M.2)).
    data_obsmix_1                      Ensemble de données simulées pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens simples (Modèle (M.4.1)).
    data_obsmix_2                      Ensemble de données simulées pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens mixtes (Modèle (M.4.2)).
    data_latmix_1                      Ensemble de données simulées pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens simples (Modèle (M.5.1)).
    data_latmix_2                      Ensemble de données simulées pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens mixtes (Modèle (M.5.2)).
    noisy_data_1                       Ensemble de données simulées pour le modèle linéaire Gaussien simple
                                       avec variables impertinentes (Modèle (M.6.1)).
    noisy_data_2                       Ensemble de données simulées pour le modèle linéaire Gaussien mixte
                                       avec variables impertinentes (Modèle (M.6.2)).
    noisy_data_latmix_1                Ensemble de données simulées pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens simples, avec variables impertinentes (Modèle (M.6.3)).
    noisy_data_latmix_2                Ensemble de données simulées pour la mixture sur l'espace latent
                                       de modèles linéaires Gaussiens mixtes, avec variables impertinentes (Modèle (M.6.4)).
    noisy_data_obsmix_1                Ensemble de données simulées pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens simples, avec variables impertinentes (Modèle (M.6.5)).
    noisy_data_obsmix_2                Ensemble de données simulées pour la mixture sur l'espace observé
                                       de modèles linéaires Gaussiens mixtes, avec variables impertinentes (Modèle (M.6.6)).
    noisy_data_spemix_1                Ensemble de données simulées pour la mixture sur seulement certaines dimensions des
                                       espaces observé et latent de modèles linéaires Gaussiens simples (Modèle (M.6.7)).
    noisy_data_spemix_2                Ensemble de données simulées pour la mixture sur seulement certaines dimensions des
                                       espaces observé et latent de modèles linéaires Gaussiens mixtes (Modèle (M.6.8)).
    ================================== ======================================================================================
'''

In [7]:
def param_lin(D,L,m1=0.0,m2=0.0,m3=0.0,s1=1.0,s2=1.0,s3=1.0,disp=False,orthog=True):
    '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien simple.
    (Modèle (M.1))
    
    Paramètres
    ----------
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes d'une matrice de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²)
        Si mis sur True, les axes principaux seront les colonnes d'une matrice de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1.0), passée par la fonction orthogonalize, puis multipliée par s1.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
        
    mu : 1-D ndarray,
        Vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
    
    sigma2 : float,
        Réel positif obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    mu = rd.normal(m2,s2**2,D)
    sigma2 = rd.normal(m3,s3**2)**2
    
    if orthog:
        W = rd.normal(m1,1.0,(D,L))
        W = s1*mbu.orthogonalize(W)
    else:
        W = rd.normal(m1,s1**2,(D,L))
    
    if disp:
        print('$W = $', W)
        print('$\mu = $', mu)
        print('$\sigma^2 = $', sigma2)
        
    return W, mu, sigma2

def data_lin(W,mu,sigma2,N,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien simple.
    (Modèle (M.1))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
        D doit être strictement supérieur à L.
        
    mu : 1-D ndarray,
        Vecteur de taille D, moyenne des observations.
    
    sigma2 : float,
        Réel positif, variance du bruit des observations.
    
    N : int,
        Nombre d'individus à simuler.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    alors y et z sont liés par la relation suivante : y = W.z + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    D,L = np.shape(W)
    Z = rd.normal(0,1,(N,L))
    Y = np.array([W@z + mu + rd.normal(0,sigma2,D) for z in Z])
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
    return Z, Y

def param_cov(D,L,C,m1=0.0,m2=0.0,m3=0.0,m4=0.0,s1=1.0,s2=1.0,s3=1.0,s4=1.0,disp=False,orthog=True):
    '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien mixte.
    (Modèle (M.2))
    
    Paramètres
    ----------
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (D,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²) et (m2,s2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (D,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1) et (m2,1), passées par la fonction orthogonalize, puis multipliées par s1 et s2.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        
    mu : 1-D ndarray,
        Vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
    
    sigma2 : float,
        Réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    mu = rd.normal(m3,s3**2,D)
    sigma2 = rd.normal(m4,s4**2)**2
    
    if orthog:
        W = rd.normal(m1,1.0,(D,L))
        V = rd.normal(m2,1.0,(D,C))
        W = s1*mbu.orthogonalize(W)
        V = s2*mbu.orthogonalize(V)
    else :
        W = rd.normal(m1,s1**2,(D,L))
        V = rd.normal(m2,s2**2,(D,C))
    
    if disp:
        print('$W = $', W)
        print('$V = $', V)
        print('$\mu = $', mu)
        print('$\sigma^2 = $', sigma2)
        
    return W, V, mu, sigma2

def data_cov(W,V,mu,sigma2,N,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien mixte.
    (Modèle (M.2))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux.
        D doit être strictement supérieur à L.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        D doit être supérieur ou égal à C.
    
    mu : 1-D ndarray,
        Vecteur de taille D, moyenne des observations.
    
    sigma2 : float,
        Réel positif, variance du bruit des observations.
    
    N : int,
        Nombre d'individus à simuler.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Z.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    alors x, y et z sont liés par la relation suivante : y = W.z + V.x + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z et x, de variance sigma2*Id.
    '''
    D1,L = np.shape(W)
    D2,C = np.shape(V)
    if D1 != D2 :
        print('Erreur de dimension sur W et V')
    else :
        D = D1
        
    Z = rd.normal(0,1,(N,L))
    E = np.ones((N,1)) @ np.array([mu]) + rd.normal(0,sigma2,(N,D))
    
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    
    Y = Z@np.transpose(W) + X@np.transpose(V) + E
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$X = $', X)
    return Z, Y, X

def sim_Lambda(D,la=1.0,p=0.1):
    pos_def = False
    while not pos_def:
        P = rd.exponential(np.sqrt(la),(D,D))
        signes = 2*rd.choice(2,(D,D)) - 1
        Q = P * signes
        full_LA = np.transpose(Q) @ Q
        
        q = np.sqrt(1-p)
        occur1 = rd.choice(2,(D,D),p=[q,1-q])
        occur2 = ((occur1 + np.transpose(occur1) + np.eye(D)).astype(bool)).astype(float)
        Lambda = occur2*full_LA
        
        SpL,P = nla.eig(Lambda)
        pos_def = np.all(SpL>0)
    return Lambda

def param_LRPSI(D,L,m1=0.0,m2=0.0,s1=0.0,s2=0.0,la=1.0,p=0.1,disp=False,orthog=True):
    
    if orthog:
        W = rd.normal(m1,1.0,(D,L))
        W = s1*mbu.orthogonalize(W)
    else :
        W = rd.normal(m1,s1**2,(D,L))
    sigma2 = rd.normal(m2,s2**2)**2
    Lambda = sim_Lambda(D,la,p)
    
    return W, Lambda, sigma2

def data_LRPSI(W,Lambda,N,sigma2,disp=False):
    D,L = np.shape(W)
    D1,D2 = np.shape(Lambda)
    if D1 != D2 or D1 != D :
        print('Erreur de dimension sur W et Lambda')
        
    Z = rd.normal(0,1,(N,L))
    E = rd.normal(0,sigma2,(N,D))
    X = rd.multivariate_normal(np.zeros(D),nla.inv(Lambda),N)
    Y = Z@np.transpose(W) + X + E
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$X = $', X)
        
    return Z, Y, X

def param_obsmix_1(K,D,L,m1=0.0,m2=0.0,m3=0.0,s1=1.0,s2=1.0,s3=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur l'espace observé de modèles linéaires Gaussiens simples.
    (Modèle (M.4.1))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des moyennes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération des variances.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des moyennes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération des variances.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes de matrices de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²)
        Si mis sur True, les axes principaux seront les colonnes de matrices de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1.0), passée par la fonction orthogonalize, puis multipliée par s1.
        Mis sur True par défaut.
        
    Renvois
    -------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    thetas = [param_lin(D,L,m1,m2,m3,s1,s2,s3,disp,orthog) for k in range(K)]
    return thetas

def param_obsmix_2(K,D,L,C,m1=0.0,m2=0.0,m3=0.0,m4=0.0,s1=1.0,s2=1.0,s3=1.0,s4=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur l'espace observé de modèles linéaires Gaussiens mixtes.
    (Modèle (M.4.2))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes de matrices de taille (D,L) et des matrices de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²) et (m2,s2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes de matrices de taille (D,L) et des matrices de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1) et (m2,1), passées par la fonction orthogonalize, puis multipliées par s1 et s2.
        Mis sur True par défaut.
        
    Renvois
    -------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster correspondant.
            - V est la matrice de taille (D,C) d'effets fixes du cluster correspondant.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    thetas = [param_cov(D,L,C,m1,m2,m3,m4,s1,s2,s3,s4,disp,orthog) for k in range(K)]
    return thetas

def param_obsmix_3(K,D,L,m1=0.0,m2=0.0,s1=0.0,s2=0.0,la=1.0,p=0.1,disp=False,orthog=True):
    thetas = [param_LRPSI(D,L,m1,m2,s1,s2,la,p,disp,orthog) for k in range(K)]
    return thetas

def param_latmix_1(K,D,L,m2=0.0,m3=0.0,s2=1.0,s3=1.0,m_glob1=0.0,m_glob2=0.0,m_glob3=0.0,s_glob2=1.0,s_glob3=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur l'espace latent de modèles linéaires Gaussiens simples
    (Modèle (M.5.1)).
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    m2 : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents.
        Mis sur 0.0 par défaut.
        
    s2 : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    m_glob1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m_glob2 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m_glob3 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s_glob2 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s_glob3 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes d'une matrice de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1).
        Si mis sur True, les axes principaux seront les colonnes d'une matrice de taille (D,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1), passée par la fonction orthogonalize.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob2,s_glob2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m_glob3,s_glob3²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille L dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    eta = param_lin(D,L,m_glob1,m_glob2,m_glob3,1.0,s_glob2,s_glob3,disp,orthog)
    thetas = [param_lin(L,L,0.0,m2,m3,1.0,s2,s3,disp,orthog)[1:] for k in range(K)]
    return eta, thetas

def param_latmix_2(K,D,L,C,m3=0.0,m4=0.0,s3=1.0,s4=1.0,m_glob1=0.0,m_glob2=0.0,m_glob3=0.0,m_glob4=0.0,s_glob2=1.0,s_glob3=1.0,s_glob4=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur l'espace latent de modèles linéaires Gaussiens mixtes, avec variables impertinentes.
    (Modèle (M.6.4))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    m_glob1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m_glob2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m_glob3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m_glob4 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s_glob2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s_glob3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s_glob4 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (D,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1) et (m_glob2,s_glob2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (D,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1) et (m_glob2,1), passées par la fonction orthogonalize, puis multipliées par 1.0 et s_glob2.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,V,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux.
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob3,s_glob3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m_glob4,s_glob4²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille L dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    eta = param_cov(D,L,C,m_glob1,m_glob2,m_glob3,m_glob4,1.0,s_glob2,s_glob3,s_glob4,disp,orthog)
    thetas = [param_lin(L,L,0.0,m3,m4,1.0,s3,s4,disp,orthog)[1:] for k in range(K)]
    return eta, thetas

def param_latmix_3(K,D,L,m2=0.0,m3=0.0,s2=1.0,s3=1.0,m_glob1=0.0,m_glob2=0.0,s_glob2=1.0,la=1.0,p=0.1,disp=False,orthog=True):
    eta = param_LRPSI(D,L,m_glob1,m_glob2,1.0,s_glob2,la,p,disp,orthog)
    thetas = [param_lin(L,L,0.0,m2,m3,1.0,s2,s3,disp,orthog)[1:] for k in range(K)]
    return eta, thetas

def sim_omega(N,K,N_min=2):
    '''Simule d'un vecteur de taille N rempli d'entiers compris entre 0 et K-1.
    
    Paramètres
    ----------
    N : int,
        Taille du vecteur à simuler.
    
    K : int,
        Nombre de clusters à simuler.
        Doit être strictement inférieur à N.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur renvoyé.
        Mis sur 2 par défaut.
        K*N_min doit être inférieur ou égal à N.
    
    Renvois
    -------
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    '''
    
    if N < K*N_min:
        K = int(N/N_min)
    
    nb_random = N - K*N_min
    random_part = rd.choice(K,nb_random)
    sorted_omega = np.concatenate([np.concatenate([k*np.ones(N_min) for k in range(K)]),random_part])
    omega = rd.permutation(sorted_omega)
    return omega.astype(int)

def data_obsmix_1(thetas,N,N_min=0,disp=False):
    '''Simule un jeu de données selon la mixture sur l'espace observé de modèles linéaires Gaussiens simples.
    (Modèle (M.4.1))
    
    Paramètres
    ----------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne de chaque cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations de chaque cluster.
        D doit être strictement supérieur à L.
        
    N : int, optional,
        Nombre d'individus à simuler.
    
    N_min : int,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur L+1 par défaut.
        N doit être strictement supérieur à K*(L+1) et K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    k est le n-ième coefficient de omega, et [W,mu,sigma2] est k-ième ensemble de paramètres de la liste thetas,
    alors y et z sont liés par la relation suivante : y = W.z + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    K = len(thetas)
    D,L = np.shape(thetas[0][0])
    
    omega = sim_omega(N,K,int(max(L+1,N_min)))
    K = int(max(omega)) + 1
    
    Z = rd.normal(0,1,(N,L))
    Y = np.array([thetas[omega[n]][0]@Z[n] + thetas[omega[n]][1] + rd.normal(0,thetas[omega[n]][2],D) for n in range(N)])
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\omega = $', omega)
    return Z, omega, Y

def data_obsmix_2(thetas,N,N_min=0,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon la mixture sur l'espace observé de modèles linéaires Gaussiens mixtes.
    (Modèle (M.4.2)).
    
    Paramètres
    ----------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux du cluster correspondant.
            - V est la matrice de taille (D,C) d'effets fixes du cluster correspondant.
            - mu est un vecteur de taille D, supposé être la moyenne de chaque cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations de chaque cluster.
        D doit être strictement supérieur à L et supérieur ou égal à C.
        
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur le max(L+1,C+1) par défaut.
        N doit être strictement supérieur à K*(L+1), K*(C+1) et K*N_min.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Z.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    k est le n-ième coefficient de omega, et [W,V,mu,sigma2] est k-ième ensemble de paramètres de la liste thetas,
    alors x, y et z sont liés par la relation suivante : y = W.z + V.x + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z et x, de variance sigma2*Id.
    '''
    K = len(thetas)
    
    D1,L = np.shape(thetas[0][0])
    D2,C = np.shape(thetas[0][1])
    
    omega = sim_omega(N,K,int(max(L+1,C+1,N_min)))
    K = int(max(omega)) + 1
    
    if D1 != D2 :
        print('Erreur de dimension sur W et V')
    else :
        D = D1
    
    Z = rd.normal(0,1,(N,L))
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    
    Y = np.array([thetas[omega[n]][0]@Z[n] + thetas[omega[n]][1]@X[n] + thetas[omega[n]][2] + rd.normal(0,thetas[omega[n]][3],D) for n in range(N)])
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
        
    return Z, omega, Y, X

def data_obsmix_3(thetas,N,N_min=0,disp=False):
    
    K = len(thetas)
    
    D,L = np.shape(thetas[0][0])
    D1,D2 = np.shape(thetas[0][1])
    if D1 != D2 or D1 != D :
        print('Erreur de dimension sur W et Lambda')
    
    omega = sim_omega(N,K,int(max(L+1,N_min)))
    K = int(max(omega)) + 1
    
    Z = rd.normal(0,1,(N,L))
    X = np.array([rd.multivariate_normal(np.zeros(D),nla.inv(thetas[omega[n]][1])) for n in range(N)])
    Y = np.array([thetas[omega[n]][0]@Z[n] + X[n] + rd.normal(0,thetas[omega[n]][2],D) for n in range(N)])
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
        
    return Z, omega, Y, X

def data_latmix_1(eta,thetas,N,N_min=2,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien mixte sur l'espace latent.
    (Modèle (M.5.1))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
        D doit être strictement supérieur à L.
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    alors y et z sont liés par la relation suivante : y = W.z + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    W, mu, sigma2 = eta
    D,L = np.shape(W)
    K = len(thetas)
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,L))
    Z = np.zeros((N,L))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Z[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
        
    noise = rd.normal(0,sigma2,(N,D))
    Y = Z@np.transpose(W) + mu + noise
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\omega = $', omega)
    
    return Z, omega, Y

def data_latmix_2(eta,thetas,N,N_min=2,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien mixte sur l'espace latent, avec covariables.
    (Modèle (M.5.2))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (D,L) dont les colonnes sont les axes principaux.
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille D supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
        D doit être strictement supérieur à L et supérieur ou égal à C.
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    alors x, y et z sont liés par la relation suivante : y = W.z + V.x + mu + epsilon[n],
    où epsilon[n] est un vecteur gaussien centré, indépendant de z et x, de variance sigma2*Id.
    '''
    W, V, mu, sigma2 = eta
    D,L = np.shape(W)
    D2,C = np.shape(V)
    K = len(thetas)
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,L))
    Z = np.zeros((N,L))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Z[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
        
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    noise = rd.normal(0,sigma2,(N,D))
    
    Y = Z@np.transpose(W) + X@np.transpose(V) + mu + noise
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
    
    return Z, omega, Y, X

def data_latmix_3(eta,thetas,N,N_min=2,Sigma_X=None,disp=False):
    
    W, Lambda, mu, sigma2 = eta
    D,L = np.shape(W)
    D1,D2 = np.shape(Lambda)
    K = len(thetas)
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,L))
    Z = np.zeros((N,L))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Z[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
        
    X = np.array([rd.multivariate_normal(np.zeros(D),nla.inv(Lambda)) for n in range(N)])
    noise = rd.normal(0,sigma2,(N,D))
    
    Y = Z@np.transpose(W) + X + mu + noise
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
    
    return Z, omega, Y, X

def noisy_param_1(D,L,U,m1=0.0,m2=0.0,m3=0.0,s1=1.0,s2=1.0,s3=1.0,disp=False,orthog=True):
    '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien simple avec variables impertinentes.
    (Modèle (M.6.1))
    
    Paramètres
    ----------
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    U : int,
        Nombre de variables pertinentes de l'espace observé.
        Doit être strictement comprsi entre L et D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes d'une matrice de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²)
        Si mis sur True, les axes principaux seront les colonnes d'une matrice de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1.0), passée par la fonction orthogonalize, puis multipliée par s1.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
        
    mu : 1-D ndarray,
        Vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
    
    sigma2 : float,
        Réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    mu = rd.normal(m2,s2**2,D)
    sigma2 = rd.normal(m3,s3**2)**2
    
    if orthog:
        W = rd.normal(m1,1.0,(U,L))
        W = s1*mbu.orthogonalize(W)
    else:
        W = rd.normal(m1,s1**2,(U,L))
    
    if disp:
        print('$W = $', W)
        print('$\mu = $', mu)
        print('$\sigma^2 = $', sigma2)
        
    return W, mu, sigma2

def noisy_data_1(W,mu,sigma2,N,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien avec variables impertinentes.
    (Modèle (M.6.1))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
        U doit être strictement supérieur à L.
        
    mu : 1-D ndarray,
        Vecteur de taille D, moyenne des observations.
        D doit être strictement supérieur à U.
    
    sigma2 : float,
        Réel positif, variance du bruit des observations.
    
    N : int,
        Nombre d'individus à simuler.
    
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    et R est la matrice ra_matrix(iota), alors y et z sont liés par la relation suivante :
    y = R.Concatenate(W.z,0_{D-U}) + mu + epsilon[n], où 0_{D-U} est le vecteur nul de taille D-U,
    et epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    U,L = np.shape(W)
    D = len(mu)
    Z = rd.normal(0,1,(N,L))
    Y_prov = Z@np.transpose(W)
    iota_prov = np.concatenate([np.ones(U), np.zeros(D-U)]).astype(int)
    
    noise = rd.normal(0,sigma2,(N,D))
    iota = rd.perumtation(iota_prov)
    Y = restit(Y_prov,iota) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\iota = $', iota)
    return Z, Y, iota

def noisy_param_2(D,L,C,U,m1=0.0,m2=0.0,m3=0.0,m4=0.0,s1=1.0,s2=1.0,s3=1.0,s4=1.0,disp=False,orthog=True):
     '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien mixte avec variables impertinentes.
     (Modèle (M.6.2))
    
    Paramètres
    ----------
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    U : int,
        Nombre de variables pertinentes de l'espace observé.
        Doit être strictement compris entre L et D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (U,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²) et (m2,s2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (U,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1) et (m2,1), passées par la fonction orthogonalize, puis multipliées par s1 et s2.
        Mis sur True par défaut.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        
    mu : 1-D ndarray,
        Vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
    
    sigma2 : float,
        Réel positif obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    mu = rd.normal(m3,s3**2,D)
    sigma2 = rd.normal(m4,s4**2)**2
    
    if orthog:
        W = rd.normal(m1,1.0,(U,L))
        V = rd.normal(m2,1.0,(D,C))
        W = s1*mbu.orthogonalize(W)
        V = s2*mbu.orthogonalize(V)
    else:
        W = rd.normal(m1,s1**2,(U,L))
        V = rd.normal(m2,s2**2,(D,C))
    
    if disp:
        print('$W = $', W)
        print('$V = $', V)
        print('$\mu = $', mu)
        print('$\sigma^2 = $', sigma2)
        
    return W, V, mu, sigma2

def noisy_data_2(W,V,mu,sigma2,N,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien avec covariables et variables impertinentes.
    (Modèle (M.6.2))
    
    Paramètres
    ----------
    W : 2-D ndarray,
        Matrice de taille (U,L) dont les colonnes sont les axes principaux.
        U doit être supérieur ou égal à L.
    
    V : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes.
        D doit être supérieur ou égal à C, et strictement supérieur à U.
    
    mu : 1-D ndarray,
        Vecteur de taille D, moyenne des observations.
    
    sigma2 : float,
        Réel positif, variance du bruit des observations.
    
    N : int,
        Nombre d'individus à simuler.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Z.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    et R est la matrice ra_matrix(iota), alors x, y et z sont liés par la relation suivante :
    y = R.Concatenate(W.z,0_{D-U}) + V.x + mu + epsilon[n], où 0_{D-U} est le vecteur nul de taille D-U,
    et epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    U,L = np.shape(W)
    D,C = np.shape(V)
        
    Z = rd.normal(0,1,(N,L))
    
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    
    Y_prov = Z@np.transpose(W)
    iota_prov = np.concatenate([np.ones(U), np.zeros(D-U)]).astype(int)
    
    noise = rd.normal(0,sigma2,(N,D))
    iota = rd.perumtation(iota_prov)
    Y = restit(Y_prov,iota) + X@np.transpose(V) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$X = $', X)
        print('$\iota = $', iota)
        
    return Z, Y, X, iota

def noisy_param_obsmix_1(K,D,L,U,m1=0.0,m2=0.0,m3=0.0,s1=1.0,s2=1.0,s3=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour le modèle linéaire Gaussien mixte sur l'espace des observations, avec variables impertinentes.
    (Modèle (M.6.5))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    U : int,
        Nombre de variables pertinentes de l'espace observé.
        Doit être strictement compris entre L et D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des moyennes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération des variances.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des moyennes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération des variances.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes de matrices de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²)
        Si mis sur True, les axes principaux seront les colonnes de matrices de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1.0), passée par la fonction orthogonalize, puis multipliée par s1.
        Mis sur True par défaut.
        
    Renvois
    -------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    thetas = [noisy_param_1(D,L,U,m1,m2,m3,s1,s2,s3,disp,orthog) for k in range(K)]
    return thetas

def noisy_param_obsmix_2(K,D,L,C,U,m1=0.0,m2=0.0,m3=0.0,m4=0.0,s1=1.0,s2=1.0,s3=1.0,s4=1.0,disp=False,orthog=True):
    '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien mixte sur l'espace observé, avec covariables et variables impertinentes.
    (Modèle (M.6.6))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    U : int,
        Nombre de dimensions pertinentes de l'espace observé.
        Doit être strictement compris entre L et D.
    
    m1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 0.0 par défaut.
    
    s1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes de matrices de taille (U,L) et des matrices de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,s1²) et (m2,s2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes de matrices de taille (U,L) et des matrices de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1,1) et (m2,1), passées par la fonction orthogonalize, puis multipliées par s1 et s2.
        Mis sur True par défaut.
        
    Renvois
    -------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,V,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster correspondant.
            - V est la matrice de taille (D,C) d'effets fixes du cluster correspondant.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    thetas = [noisy_param_2(D,L,C,U,m1,m2,m3,m4,s1,s2,s3,s4,disp,orthog) for k in range(K)]
    return thetas

def noisy_param_latmix_1(K,D,L,U,m2=0.0,m3=0.0,s2=1.0,s3=1.0,m_glob1=0.0,m_glob2=0.0,m_glob3=0.0,s_glob2=1.0,s_glob3=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur l'espace latent de modèles linéaires Gaussiens simples, avec variables impertinentes.
    (Modèle (M.6.3))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    U : int,
        Nombre de dimensions pertinentes de l'espace observé.
        Doit être strictement compris entre L et D.
    
    m2 : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    m3 : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents.
        Mis sur 0.0 par défaut.
        
    s2 : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    m_glob1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m_glob2 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m_glob3 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s_glob2 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s_glob3 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux seront les colonnes d'une matrice de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1).
        Si mis sur True, les axes principaux seront les colonnes d'une matrice de taille (U,L) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1), passée par la fonction orthogonalize.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob2,s_glob2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m_glob3,s_glob3²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille L dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2,s2²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3,s3²).
    '''
    eta = noisy_param_1(D,L,U,m_glob1,m_glob2,m_glob3,1.0,s_glob2,s_glob3,disp,orthog)
    thetas = [param_lin(L,L,0.0,m2,m3,1.0,s2,s3,disp,orthog)[1:] for k in range(K)]
    return eta, thetas

def noisy_param_latmix_2(K,D,L,C,U,m3=0.0,m4=0.0,s3=1.0,s4=1.0,m_glob1=0.0,m_glob2=0.0,m_glob3=0.0,m_glob4=0.0,s_glob2=1.0,s_glob3=1.0,s_glob4=1.0,disp=False,orthog=True):
    '''Simule un ensemble de paramètres pour le modèle linéaire Gaussien mixte sur l'espace latent, avec covariables et variables et variables impertinentes.
    (Modèle (M.6.4))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    D : int,
        Nombre de dimensions de l'espace observé.
    
    L : int,
        Nombre de dimensions de l'espace latent.
        Doit être strictement inférieur à D-1.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à D.
    
    U : int,
        Nombre de dimensions pertinentes de l'espace observé.
        Doit être strictement compris entre L et D.
    
    m3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    m4 : float, optional,
        Paramètre influent sur la génération de la variance des vecteurs latents.
        Mis sur 0.0 par défaut.
    
    s3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    s4 : float, optional,
        Paramètre influent sur la génération de la variance des vecteurs latents.
        Mis sur 1.0 par défaut.
    
    m_glob1 : float, optional,
        Paramètre influent sur la génération des axes principaux.
        Mis sur 0.0 par défaut.
    
    m_glob2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m_glob3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m_glob4 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s_glob2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s_glob3 : float, optional,
        Paramètre influent sur la génération de la moyenne des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s_glob4 : float, optional,
        Paramètre influent sur la génération de la variance du bruit des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (U,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1) et (m_glob2,s_glob2²).
        Si mis sur True, les axes principaux et les effets fixes seront respectivement les colonnes d'une matrice de taille (U,L) et une matrice de taille (D,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob1,1) et (m_glob2,1), passées par la fonction orthogonalize, puis multipliées par 1.0 et s_glob2.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,V,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille D dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m_glob3,s_glob3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m_glob4,s_glob4²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille L dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3,s3²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4,s4²).
    '''
    eta = noisy_param_2(D,L,C,U,m_glob1,m_glob2,m_glob3,m_glob4,1.0,s_glob2,s_glob3,s_glob4,disp,orthog)
    thetas = [param_lin(L,L,0.0,m3,m4,1.0,s3,s4,disp,orthog)[1:] for k in range(K)]
    return eta, thetas

def noisy_data_obsmix_1(thetas,N,N_min=0,disp=False):
    '''Simule un jeu de données selon la mixture sur l'espace observé de modèles linéaires Gaussiens simples, avec variables impertinentes.
    (Modèle (M.6.5)).
    
    Paramètres
    ----------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne de chaque cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations de chaque cluster.
        On doit avoir D > U > L.
        
    N : int, optional,
        Nombre d'individus à simuler.
    
    N_min : int,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur L+1 par défaut.
        N doit être strictement supérieur à K*(L+1) et K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) dont chaque ligne contient U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    k est le n-ième coefficient de omega, [W,mu,sigma2] est k-ième ensemble de paramètres de la liste thetas,
    iota est la k-ième ligne de iotas et R est la matrice ra_matrix(iota), alors y et z sont liés par la
    relation suivante : y = R.Concatenate(W.z,0_{D-U}) + mu + epsilon[n], où 0_{D-U} est le vecteur nul de
    taille D-U, et epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    K = len(thetas)
    U,L = np.shape(thetas[0][0])
    D = len(thetas[0][1])
    
    omega = sim_omega(N,K,int(max(L+1,N_min)))
    K = int(max(omega)) + 1
    
    Z = rd.normal(0,1,(N,L))
    iotas_prov = np.concatenate([np.ones((K,U)),np.zeros((K,D-U))],axis=1).astype(int)
    iotas = np.array([rd.permutation(iota) for iota in iotas_prov])
    
    Y = drr.FS_mixrec1(thetas,Z,omega,iotas) + np.array([rd.normal(0,thetas[omega[n]][2],D) for n in range(N)])
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\omega = $', omega)
        print('$\iotas = $', iotas)
        
    return Z, omega, Y, iotas

def noisy_data_obsmix_2(thetas,N,N_min=0,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon la mixture sur l'espace observé de modèles linéaires Gaussiens mixtes, avec variables impertinentes.
    (Modèle (M.6.6)).
    
    Paramètres
    ----------
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster correspondant.
            - V est la matrice de taille (D,C) d'effets fixes du cluster correspondant.
            - mu est un vecteur de taille D, supposé être la moyenne de chaque cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations de chaque cluster.
        On doit avoir D > U > L, et C doit être inférieur ou égal à D.
        
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur max(L+1,C+1) par défaut.
        N doit être strictement supérieur à K*(L+1), K*(C+1) et K*N_min.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Z.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) dont chaque ligne contient U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    k est le n-ième coefficient de omega, [W,V,mu,sigma2] est k-ième ensemble de paramètres de la liste thetas,
    iota est la k-ième ligne de iotas et R est la matrice ra_matrix(iota), alors x, y et z sont liés par
    la relation suivante : y = R.Concatenate(W.z,0_{D-U}) + V.x + mu + epsilon[n], où 0_{D-U} est le vecteur nul
    de taille D-U, et epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    K = len(thetas)
    U,L = np.shape(thetas[0][0])
    D,C = np.shape(thetas[0][1])
    
    omega = sim_omega(N,K,int(max(L+1,C+1,N_min)))
    K = int(max(omega)) + 1
    
    Z = rd.normal(0,1,(N,L))
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    
    iotas_prov = np.concatenate([np.ones((K,U)),np.zeros((K,D-U))],axis=1).astype(int)
    iotas = np.array([rd.permutation(iota) for iota in iotas_prov])
    
    Y = drr.FS_mixrec2(thetas,Z,X,omega,iotas) + np.array([rd.normal(0,thetas[omega[n]][3],D) for n in range(N)])
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
        print('$\iotas = $', iotas)
        
    return Z, omega, Y, X, iotas

def noisy_data_latmix_1(eta,thetas,N,N_min=2,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien mixte sur l'espace latent, avec variables impertinentes.
    (Modèle (M.6.3))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
        On doit avoir : D > U > L.
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si y et z sont les n-ièmes lignes respectives de Y et Z,
    et R est la matrice ra_matrix(iota), alors y et z sont liés par la relation suivante :
    y = R.Concatenate(W.z,0_{D-U}) + mu + epsilon[n], où 0_{D-U} est le vecteur nul de taille D-U,
    et epsilon[n] est un vecteur gaussien centré, indépendant de z, de variance sigma2*Id.
    '''
    W, mu, sigma2 = eta
    U,L = np.shape(W)
    D = len(mu)
    K = len(thetas)
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,L))
    Z = np.zeros((N,L))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Z[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
        
    Y_prov = Z@np.transpose(W)
    iota_prov = np.concatenate([np.ones(U), np.zeros(D-U)]).astype(int)
    
    noise = rd.normal(0,sigma2,(N,D))
    iota = rd.permutation(iota_prov)
    
    Y = drr.restit(Y_prov,iota) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\omega = $', omega)
        print('$\iota = $', iota)
    
    return Z, omega, Y, iota

def noisy_data_latmix_2(eta,thetas,N,N_min=2,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon le modèle linéaire Gaussien mixte sur l'espace latent, avec covariables.
    (Modèle (M.6.4))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2], où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille D supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
        On doit avoir : D > U > L, et C doit être inférieur ou égal à D.
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    Sigma_X : 2-D ndarray, optional,
        Matrice carrée d'ordre C, symétrique positive, de covariance des covariables.
        Si mis sur None, prend comme valeur la matrice identité d'ordre C.
        Mis sur None par défaut.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents simulés.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés obtenus.
        
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Z.
        
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    
    Pour tout n entre 0 et N-1, si x, y et z sont les n-ièmes lignes respectives de X, Y et Z,
    et R est la matrice ra_matrix(iota), alors x, y et z sont liés par la relation suivante :
    y = R.Concatenate(W.z,0_{D-U}) + V.x + mu + epsilon[n], où 0_{D-U} est le vecteur nul de taille D-U,
    et epsilon[n] est un vecteur gaussien centré, indépendant de z et x, de variance sigma2*Id.
    '''
    W, V, mu, sigma2 = eta
    U,L = np.shape(W)
    D,C = np.shape(V)
    K = len(thetas)
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,L))
    Z = np.zeros((N,L))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Z[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
        
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
        
    noise = rd.normal(0,sigma2,(N,D))
    Y_prov = np.concatenate([Z@np.transpose(W),np.zeros((N,D-U))],axis=1)
    iota_prov = np.concatenate([np.ones((1,U)),np.zeros((1,D-U))],axis=1).astype(int)
    iota = rd.permutation(iota_prov)
    
    Y = drr.restit(Y_prov,iota) + X@np.transpose(V) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
        print('$\iota = $', iota)
    
    return Z, omega, Y, X, iota

def noisy_param_spemix_1(K,Dv,Du,Lv,Lu,m1u=0.0,m1v=0.0,m2u=0.0,m2v=0.0,m2t=0.0,m3u=0.0,m3v=0.0,m3t=0.0,s1u=1.0,s1v=1.0,s2u=1.0,s2v=1.0,s2t=1.0,s3u=1.0,s3v=1.0,s3t=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur seulement certaines dimensions des espaces observé et latent de modèles linéaires Gaussiens simples.
    (Modèle (M.6.7))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    Dv : int,
        Nombre de dimensions de l'espace observé sur lesquelles le modèle est mixte.
    
    Du : int,
        Nombre de dimensions de l'espace observé sur lesquelles le modèle n'est pas mixte.
    
    Lv : int,
        Nombre de dimensions de l'espace latent sur lesquelles le modèle est mixte.
        Doit être strictement inférieur à Dv.
    
    Lu : int,
        Nombre de dimensions de l'espace latent sur lesquelles le modèle n'est pas mixte.
        Doit être strictement inférieur à Du.
    
    m1u : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "non-mixtes".
        Mis sur 0.0 par défaut.
        
    m1v : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "mixtes".
        Mis sur 0.0 par défaut.
    
    m2u : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 0.0 par défaut.
        
    m2v : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi dépend du cluster.
        Mis sur 0.0 par défaut.
    
    m2t : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m3u : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 0.0 par défaut.
        
    m3v : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi dépend du cluster.
        Mis sur 0.0 par défaut.
    
    m3t : float, optional,
        Paramètre influent sur la génération des variances des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s1u : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "non-mixtes".
        Mis sur 1.0 par défaut.
        
    s1v : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "mixtes".
        Mis sur 1.0 par défaut.
    
    s2u : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 1.0 par défaut.
        
    s2v : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi dépend du cluster.
        Mis sur 1.0 par défaut.
    
    s2t : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s3u : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 1.0 par défaut.
        
    s3v : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi dépend du cluster.
        Mis sur 1.0 par défaut.
    
    s3t : float, optional,
        Paramètre influent sur la génération des variances des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux des dimensions "mixtes" et "non-mixtes" seront respectivement les colonnes d'une matrice de taille (Dv,Lv) et (Du,Lu) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1v,s1v²) et (m1u,s1u²).
        Si mis sur True, les axes principaux des dimensions "mixtes" et "non-mixtes" seront respectivement les colonnes d'une matrice de taille (Dv,Lv) et (Du,Lu) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1v,1) et (m1u,s1), passée par la fonction orthogonalize puis multipliée par s1v et s1u.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - mu est un vecteur de taille Dv+Du dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2t,s2t²).
            - nu est un vecteur de taille Lu dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2u,s2u²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3t,s3t²).
            - tau2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3u,s3u²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille Lv dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m2v,s2v²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m3v,s3v²).
    '''
    D = Du+Dv
    mu = rd.normal(m2t,s2t**2,D)
    nu = rd.normal(m2u,s2u**2,Lu)
    sigma2 = rd.normal(m3t,s3t**2)**2
    tau2 = rd.normal(m3u,s3u**2)**2
    
    thetas = [[] for k in range(K)]    
    for k in range(K):
        mu_k = rd.normal(m2v,s2v**2,Lv)
        sigma2_k = rd.normal(m3v,s3v**2)**2
        thetas[k] = [mu_k,sigma2_k]    
    
    if orthog:
        Wu = rd.normal(m1u,1.0,(Du,Lu))
        Wv = rd.normal(m1v,1.0,(Dv,Lv))
        Wu = s1u*mbu.orthogonalize(Wu)
        Wv = s1v*mbu.orthogonalize(Wv)
    else:
        Wu = rd.normal(m1u,s1u**2,(Du,Lu))
        Wv = rd.normal(m1v,s1v**2,(Dv,Lv))
    
    eta = [Wv,Wu,mu,nu,sigma2,tau2]
    
    if disp:
        print("eta =",eta)
        print("thetas =",thetas)
        
    return eta, thetas

def noisy_param_spemix_2(K,Dv,Du,Lv,Lu,C,m1u=0.0,m1v=0.0,m2=0.0,m3u=0.0,m3v=0.0,m3t=0.0,m4u=0.0,m4v=0.0,m4t=0.0,s1u=1.0,s1v=1.0,s2=1.0,s3u=1.0,s3v=1.0,s3t=1.0,s4u=1.0,s4v=1.0,s4t=1.0,disp=False,orthog=True):
    '''Simule K ensembles de paramètres pour la mixture sur seulement certaines dimensions des espaces observé et latent de modèles linéaires Gaussiens mixtes.
    (Modèle (M.6.8))
    
    Paramètres
    ----------
    K : int,
        Nombre de clusters.
    
    Dv : int,
        Nombre de dimensions de l'espace observé sur lesquelles le modèle est mixte.
    
    Du : int,
        Nombre de dimensions de l'espace observé sur lesquelles le modèle n'est pas mixte.
    
    Lv : int,
        Nombre de dimensions de l'espace latent sur lesquelles le modèle est mixte.
        Doit être strictement inférieur à Dv.
    
    Lu : int,
        Nombre de dimensions de l'espace latent sur lesquelles le modèle n'est pas mixte.
        Doit être strictement inférieur à Du.
    
    C : int,
        Nombre de dimensions des vecteurs de covariables.
        Doit être inférieur ou égal à Dv+Du.
    
    m1u : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "non-mixtes".
        Mis sur 0.0 par défaut.
        
    m1v : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "mixtes".
        Mis sur 0.0 par défaut.
    
    m2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 0.0 par défaut.
    
    m3u : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 0.0 par défaut.
        
    m3v : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi dépend du cluster.
        Mis sur 0.0 par défaut.
    
    m3t : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    m4u : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 0.0 par défaut.
        
    m4v : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi dépend du cluster.
        Mis sur 0.0 par défaut.
    
    m4t : float, optional,
        Paramètre influent sur la génération des variances des vecteurs observés.
        Mis sur 0.0 par défaut.
    
    s1u : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "non-mixtes".
        Mis sur 1.0 par défaut.
        
    s1v : float, optional,
        Paramètre influent sur la génération des axes principaux des dimensions "mixtes".
        Mis sur 1.0 par défaut.
    
    s2 : float, optional,
        Paramètre influent sur la génération des effets fixes.
        Mis sur 1.0 par défaut.
    
    s3u : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 1.0 par défaut.
        
    s3v : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs latents dont la loi dépend du cluster.
        Mis sur 1.0 par défaut.
    
    s3t : float, optional,
        Paramètre influent sur la génération des moyennes des vecteurs observés.
        Mis sur 1.0 par défaut.
    
    s4u : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi ne dépend pas du cluster.
        Mis sur 1.0 par défaut.
        
    s4v : float, optional,
        Paramètre influent sur la génération des variances des vecteurs latents dont la loi dépend du cluster.
        Mis sur 1.0 par défaut.
    
    s4t : float, optional,
        Paramètre influent sur la génération des variances des vecteurs observés.
        Mis sur 1.0 par défaut.
        
    disp : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les paramètres obtenus.
        Mis sur False par défaut.
    
    orthog : bool, optional,
        Si mis sur False, les axes principaux des dimensions "mixtes" et "non-mixtes" et les effets fixes seront respectivement les colonnes d'une matrice de taille (Dv,Lv), (Du,Lu) et (Dv+Du,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1v,s1v²), (m1u,s1u²) et (m2,s2²).
        Si mis sur True, les axes principaux des dimensions "mixtes" et "non-mixtes" et les effets fixes seront respectivement les colonnes d'une matrice de taille (Dv,Lv), (Du,Lu) et (Dv+Du,C) dont les coefficients seront des variables aléatoires gaussiennes i.i.d. de paramètres (m1v,1), (m1u,s1) et (m2,1) passée par la fonction orthogonalize puis multipliée par s1v, s1u et s2.
        Mis sur True par défaut.
        
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,V,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille Dv+Du dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3t,s3t²).
            - nu est un vecteur de taille Lu dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3u,s3u²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4t,s4t²).
            - tau2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4u,s4u²).
    
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu,sigma2], où :
            - mu est un vecteur de taille Lv dont les coefficients sont des variables aléatoires gaussiennes i.i.d. de paramètres (m3v,s3v²).
            - sigma2 est un réel positif, obtenu en élevant au carré une variable aléatoire gaussienne de paramètres (m4v,s4v²).
    '''
    D = Du+Dv
    
    mu = rd.normal(m3t,s3t**2,D)
    nu = rd.normal(m3u,s3u**2,Lu)
    sigma2 = rd.normal(m4t,s4t**2)**2
    tau2 = rd.normal(m4u,s4u**2)**2
    
    thetas = [[] for k in range(K)]    
    for k in range(K):
        mu_k = rd.normal(m3v,s3v**2,Lv)
        sigma2_k = rd.normal(m4v,s4v**2)**2
        thetas[k] = [mu_k,sigma2_k]    
    
    if orthog:
        Wu = rd.normal(m1u,1.0,(Du,Lu))
        Wv = rd.normal(m1v,1.0,(Dv,Lv))
        V = rd.normal(m2,1.0,(D,C))
        Wu = s1u*mbu.orthogonalize(Wu)
        Wv = s1v*mbu.orthogonalize(Wv)
        V = s2*mbu.orthogonalize(V)
    else:
        Wu = rd.normal(m1u,s1u**2,(Du,Lu))
        Wv = rd.normal(m1v,s1v**2,(Dv,Lv))
        V = rd.normal(m2,s2**2,(D,C))
    
    eta = [Wv,Wu,V,mu,nu,sigma2,tau2]
    
    if disp:
        print("eta =",eta)
        print("thetas =",thetas)
        
    return eta, thetas

def noisy_data_spemix_1(eta,thetas,N,N_min=2,disp=False):
    '''Simule un jeu de données selon la mixture sur seulement certaines dimensions des espaces observé et latent de modèles linéaires Gaussiens simples.
    (Modèle (M.6.7)).
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas selon le cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des vecteurs observés.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas selon le cluster.
        On doit avoir Dv > Lv, et Du > Lu.
        
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents dont la loi change selon les clusters.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents dont la loi ne change pas selon les clusters.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,Dv+Du) dont les lignes sont les vecteurs observés obtenus.
    
    iota : 1-D ndarray,
        Vecteur de taille Dv+Du contenant Dv fois le nombre 1 et Du fois le nombre 0, où le nombre 1 signifie que la loi des observations sur l'axe correspondant diffère selon les clusters, et le nombre 0 signifie que la loi des observations sur l'axe correspondant ne diffère pas selon les clusters.
    
    Pour tout n entre 0 et N-1, si y, zv et zu sont les n-ièmes lignes respectives de Y, Zv et Zu,
    et R est la matrice ra_matrix(iota), alors y, zv et zu sont liés par la relation suivante :
    y = R.Concatenate(Wv.zv,Wu.zu) + mu + epsilon[n],
    et epsilon[n] est un vecteur gaussien centré, indépendant de zv et zu, de variance sigma2*Id.
    '''
    Wv, Wu, mu, nu, sigma2, tau2 = eta
    Du,Lu = np.shape(Wu)
    Dv,Lv = np.shape(Wv)
    D = len(mu)
    K = len(thetas)
    
    if D != Du + Dv:
        print("Erreur de dimensions sur Wu, Wv et mu")
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,Lv))
    Zv = np.zeros((N,Lv))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Zv[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
    
    Zu = rd.normal(nu,tau2,(N,Lu))
    
    Y_prov = np.concatenate([Zv@np.transpose(Wv),Zu@np.transpose(Wu)],axis=1)
    iota_prov = np.concatenate([np.ones(Dv), np.zeros(Du)]).astype(int)
    
    noise = rd.normal(0,sigma2,(N,D))
    iota = rd.permutation(iota_prov)
    
    Y = drr.rearg(Y_prov,iota) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$\omega = $', omega)
        print('$\iota = $', iota)
    
    return Zv, Zu, omega, Y, iota

def noisy_data_spemix_2(eta,thetas,N,N_min=2,Sigma_X=None,disp=False):
    '''Simule un jeu de données selon la mixture sur seulement certaines dimensions des espaces observé et latent de modèles linéaires Gaussiens mixtes.
    (Modèle (M.6.8))
    
    Paramètres
    ----------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,V,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas selon le cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des vecteurs observés.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas selon le cluster.
        On doit avoir : Dv > Lv et Du > Lu, et C doit être inférieur ou égal à Dv+Du.
        
    thetas : list,
        Liste à K éléments, dont chacun est un ensemble de paramètres de la forme [mu_k,sigma2_k], où :
            - mu_k est un vecteur de taille Lv, supposé être une moyenne locale de vecteurs latents dont la loi change selon le cluster.
            - sigma2_k est un réel positif, supposé être une variance locale de vecteurs latents dont la loi change selon le cluster.
           
    N : int,
        Nombre d'individus à simuler.
    
    N_min : int, optional,
        Nombre minimal d'occurences de chaque entier entre 0 et K-1 dans le vecteur omega renvoyé.
        Prend comme valeur 2 par défaut.
        N doit être strictement supérieur à K*N_min.
    
    disp : float, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche les données simulées.
        Mis sur False par défaut. 
        
    Renvois
    -------
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents dont la loi change selon les clusters.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents dont la loi ne change pas selon les clusters, indépendante de Zv.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    Y : 2-D ndarray,
        Matrice de taille (N,Dv+Du) dont les lignes sont les vecteurs observés obtenus.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont les vecteurs de covariables simulés, indépendante de Zv et Zu.
    
    iota : 1-D ndarray,
        Vecteur de taille Dv+Du contenant Dv fois le nombre 1 et Du fois le nombre 0, où le nombre 1 signifie que la loi des observations sur l'axe correspondant diffère selon les clusters, et le nombre 0 signifie que la loi des observations sur l'axe correspondant ne diffère pas selon les clusters.
    
    Pour tout n entre 0 et N-1, si y, zv et zu sont les n-ièmes lignes respectives de Y, Zv et Zu,
    et R est la matrice ra_matrix(iota), alors y, zv et zu sont liés par la relation suivante :
    y = R.Concatenate(Wv.zv,Wu.zu) + mu + epsilon[n],
    et epsilon[n] est un vecteur gaussien centré, indépendant de zv et zu, de variance sigma2*Id.
    '''
    Wv, Wu, V, mu, nu, sigma2, tau2 = eta
    Du,Lu = np.shape(Wu)
    Dv,Lv = np.shape(Wv)
    D,C = np.shape(V)
    K = len(thetas)
    
    if D != Du + Dv:
        print("Erreur de dimensions sur Wu, Wv et V")
    
    omega = sim_omega(N,K,N_min)
    K = int(max(omega)) + 1
    
    Z_orig = rd.normal(0,1,(N,Lv))
    Zv = np.zeros((N,Lv))
    for n in range(N):
        k = omega[n]
        mu_k,sigma2_k = thetas[k]
        Zv[n] = np.sqrt(sigma2_k)*Z_orig[n] + mu_k
    
    Zu = rd.normal(nu,tau2,(N,Lu))
    
    Y_prov = np.concatenate([Zv@np.transpose(Wv),Zu@np.transpose(Wu)],axis=1)
    iota_prov = np.concatenate([np.ones(Dv), np.zeros(Du)]).astype(int)
    
    noise = rd.normal(0,sigma2,(N,D))
    iota = rd.permutation(iota_prov)
    
    if type(Sigma_X) == type(None) :
        X = rd.normal(0,1,(N,C))
    else :
        X = rd.multivariate_normal(np.zeros(C),Sigma_X,N)
    
    Y = drr.rearg(Y_prov,iota) + X@np.transpose(V) + noise + mu
    
    if disp:
        print('$Z = $', Z)
        print('$Y = $', Y)
        print('$Y = $', X)
        print('$\omega = $', omega)
        print('$\iota = $', iota)
    
    return Zv, Zu, omega, Y, X, iota

# Fonctions de sélection de variables (FSF)

In [ ]:
'''======================================
    Fonctions de la sélection de variables
    ======================================
    
    ================================== ==========================================================================
    Contient les fonctions suivantes :
    ---------------------------------- --------------------------------------------------------------------------
    PPCA                               Analyse en Composante Principale, puis Sélection de Variables
                                       (Adapté pour le modèle (M.6.1)).
    RCA                                Analyse en Composante Résiduelle (méthode itérative), puis Sélection de Variables
                                       (Adapté pour le modèle (M.6.2)).
    lat1                               PPCA, puis Sélection de Variables, puis clustering
                                       (Adapté pour le modèle (M.6.3)).
    lat2                               RCA, puis Sélection de Variables, puis clustering
                                       (Adapté pour le modèle (M.6.4)).
    obs1                               Clustering, puis PPCA et Sélection de Variables appliquées sur chaque cluster
                                       (Adapté pour le modèle (M.6.5)).
    obs2                               Clustering, puis RCA et Sélection de Variables appliquées sur chaque cluster
                                       (Adapté pour le modèle (M.6.6)).
    spe1                               PPCA, puis clustering, puis Sélection de Variables
                                       (Adapté pour le modèle (M.6.7)).
    spe2                               RCA, puis clustering, puis Sélection de Variables
                                       (Adapté pour le modèle (M.6.8)).
    ================================== ==========================================================================
'''

In [15]:
def PPCA(Y,L=None,U=None):
    '''Analyse en Composante Principale, puis Sélection de Variables.
    (Adapté pour le modèle (M.6.1))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
        
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
        
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la PCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PCA.
    
    sigma2 : float,
        Estimation de la variance du bruit.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    W_hat,Z_hat,sigma2_hat = sfa.PPCA(Y,L)
    iota_hat = ufs.iotate(Y,Z_hat,U)
    
    U,L = np.shape(W_hat)
    N,D = np.shape(Y)
    Y_tilde = drr.discard(Y,iota_hat)
    
    W_hac,Z_hac,sigma2_hac = sfa.PPCA(Y_tilde,L)
    
    return W_hac,Z_hac,sigma2_hac,iota_hat

def RCA(Y,X,L,V=None,nb_steps=100,err=0.0,tempo=True,U=None):
    '''Analyse en Composante Résiduelle (méthode itérative), puis Sélection de Variables.
    (Adapté pour le modèle (M.6.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int,
        Nombre de dimensions latentes souhaité.
        
    V : 2-D ndarray, optional,
        Matrice de taille (D,C) d'effets fixes donnée en argument initial de l'algorithme itératif.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme.
        Mis sur 1000 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme.
        Mis sur True par défaut.
    
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
    
    Renvois
    -------
    W : 2-D ndarray,
        Matrice de taille (D,L) dont les colonnes sont les axes principaux obtenus par la RCA.
        
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la RCA.
        
    V_hat : 2-D ndarray,
        Matrice de taille (D,C) d'effets fixes estimée par la RCA.
    
    sigma2 : float,
        Estimation de la variance du bruit.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    W_hat, Z_hat, V_hat, sigma2_hat = sfa.ML_RCA(Y,X,L,V,nb_steps,err,tempo)
    mu_X = np.mean(X,axis=0)
    Xc = X - mu_X
    
    iota_hat = ufs.iotate(Y-Xc@np.transpose(V_hat),Z_hat,U)
    D,C = np.shape(V_hat)
    U,L = np.shape(W_hat)
    N = len(Y)
    Y_tilde = drr.discard(Y-Xc@np.transpose(V_hat),iota_hat)
    
    W_hac,Z_hac,sigma2_hac = sfa.PPCA(Y_tilde,L)
    
    return W_hac, Z_hac, V_hat, sigma2_hac, iota_hat

def lat1(Y,L=None,K=None,omega=None,fun=clf.Lapras,nb_steps=100,tempo=True,U=None):
    '''PPCA, puis Sélection de Variables, puis clustering.
    (Adapté pour le modèle (M.6.3))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D, supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N,D = np.shape(Y)
    W_prov,Z_prov,sigma2_hat = sfa.PPCA(Y,L)
    D,L = np.shape(W_prov)
    
    D_W = np.diag(np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    D_W_inv = np.diag(1/np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    Z_hat = Z_prov @ D_W
    W_hat = W_prov @ D_W_inv
    
    mu_hat = np.mean(Y,axis=0)
    
    iota_hat = ufs.iotate(Y,Z_hat,U)
    
    U = np.sum(iota_hat)
    Y_tilde = drr.discard(Y,iota_hat)
    
    W_hac,Z_hac,sigma2_hac = sfa.PPCA(Y_tilde,L)
        
    if type(omega) == type(None):
        
        if type(K) == type(None):
            K = clf.K_opt(Z_hac)
                
        omega_hat = fun(Z_hac,K,tempo=tempo)
        omega_hat = clf.K_means(Z_hac,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    tri_Z = ufc.tri(Z_hac,omega_hat)
    thetas_hac = []
    
    for k in range(K):
        z = tri_Z[k]
        thetas_k = sfa.MLE_Gauss(z)
        thetas_hac.append(thetas_k)
    
    Y_hat = drr.FS_rec2(W_hac,Z_hac,mu_hat,iota_hat)
    sigma2_hac = np.mean((Y-Y_hat)**2)
    eta_hac = W_hac,mu_hat,sigma2_hac
    
    return eta_hac, thetas_hac, Z_hac, omega_hat, iota_hat

def lat2(Y,X,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,err=0.0,tempo=True,U=None):
    '''RCA, puis Sélection de Variables, puis clustering.
    (Adapté pour le modèle (M.6.4))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int,
        Nombre de dimensions latentes souhaité.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
    
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
    
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux.
            - mu est un vecteur de taille D, supposé être la moyenne des observations.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations.
            
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N,D = np.shape(Y)
    N1,C = np.shape(X)
    W_prov, Z_prov, V_hat, sigma2_hat = sfa.ML_RCA(Y,X,L,nb_steps=nb_steps,err=err,tempo=tempo)
    
    D_W = np.diag(np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    D_W_inv = np.diag(1/np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
    Z_hat = Z_prov @ D_W
    W_hat = W_prov @ D_W_inv
    
    mu_hat = np.mean(Y,axis=0)
    mu_X = np.mean(X,axis=0)
    Xc = X - mu_X
    
    iota_hat = ufs.iotate(Y-Xc@np.transpose(V_hat),Z_hat,U)
    
    U = np.sum(iota_hat)
    Y_tilde = drr.discard(Y-Xc@np.transpose(V_hat),iota_hat)
    
    W_hac,Z_hac,sigma2_hac = sfa.PPCA(Y_tilde,L)
    
    if type(omega) == type(None):
        
        if type(K) == type(None):
            K = clf.K_opt(Z_hac)
                
        omega_hat = fun(Z_hac,K,tempo=tempo)
        omega_hat = clf.K_means(Z_hac,omega_hat,nb_steps,tempo=tempo)
    
    else:
        omega_hat = omega.copy()
        K = int(max(omega) + 1)
    
    tri_Z = ufc.tri(Z_hac,omega_hat)
    thetas_hat = []
    
    for k in range(K):
        z = tri_Z[k]
        thetas_k = sfa.MLE_Gauss(z)
        thetas_hat.append(thetas_k)
    
    Y_hat = drr.FS_rec2(W_hac,Z_hac,V_hat,X,mu_hat,iota_hat)
    sigma2_hac = np.mean((Y-Y_hat)**2)
    eta_hac = W_hac,V_hat,mu_hat,sigma2_hac
        
    return eta_hac, thetas_hat, Z_hac, omega_hat, iota_hat

def obs1(Y,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,tempo=True,U=None):
    '''Clustering, puis PPCA appliquée sur chaque cluster, puis Sélection de Variables sur chaque cluster.
    (Adapté pour le modèle (M.6.5))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    L : int,
        Nombre de dimensions de l'espace latent.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
    
    Renvois
    -------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la PPCA, cluster par cluster.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) dont chaque ligne contient U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N,D = np.shape(Y)
    thetas_hat, Z_hat, omega_hat = mfa.obs1(Y,L,K,omega,fun,nb_steps,tempo)
    
    K = len(thetas_hat)
    tri_Y = ufc.tri(Y,omega_hat)
    tri_Z = ufc.tri(Z_hat,omega_hat)
    
    iotas_hat = np.zeros((K,D)).astype(int)
    reclus = ufc.tri((np.arange(N)).astype(int),omega_hat)
    Z_hac = np.zeros((N,L))
    thetas_hac = [[] for k in range(K)]
    
    for k in range(K):
        
        Y_k = tri_Y[k]
        Z_k = tri_Z[k]
        card_k = len(Y_k)
        
        mu_k_hac = np.mean(Y_k,axis=0)
        iota_k_hat = ufs.iotate(Y_k,Z_k,U)
        iotas_hat[k] = iota_k_hat
        U = np.sum(iota_k_hat)
        
        Y_k_tilde = drr.discard(Y_k,iota_k_hat)
        W_k_hac, Z_k_hac, sigma2_k_hac = sfa.PPCA(Y_k_tilde,L)
        
        for j in range(card_k):
            Z_hac[reclus[k][j]] = Z_k_hac[j]
        
        Y_k_hac = drr.FS_rec1(W_k_hac,Z_k_hac,mu_k_hac,iota_k_hat)
        sigma2_k_hac = np.mean((Y_k-Y_k_hac)**2)
            
        thetas_hac[k] = [W_k_hac,mu_k_hac,sigma2_k_hac]
        
    return thetas_hac, Z_hac, omega_hat, iotas_hat

def obs2(Y,X,L,K=None,omega=None,fun=clf.Lapras,nb_steps=100,err=0.0,tempo=True,U=None):
    '''Clustering, puis RCA appliquée sur chaque cluster, puis Sélection de Variables sur chaque cluster.
    (Adapté pour le modèle (M.6.6))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force, cluster par cluster.
    
    L : int,
        Nombre de dimensions latentes souhaité.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme K-means.
        Mis sur 100 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s'arrête.
        Mis sur 0.0 par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    U : int, optional,
        Nombre de dimensions pertinentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de U renvoyée par la fonction U_opt.
    
    Renvois
    -------
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [W,V,mu,sigma2] où :
            - W est une matrice de taille (U,L) dont les colonnes sont les axes principaux du cluster.
            - V est la matrice de taille (D,C) d'effets fixes du cluster.
            - mu est un vecteur de taille D, supposé être la moyenne des observations du cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des observations du cluster.
            
    Z : 2-D ndarray,
        Matrice de taille (N,L) dont les lignes sont les vecteurs latents obtenus par la RCA, cluster par cluster.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iotas : 2-D ndarray,
        Matrice de taille (K,D) dont chaque ligne contient U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la dimension correspondante est pertinente, et le nombre 0 signifie qu'elle ne l'est pas.
    '''
    N,D = np.shape(Y)
    thetas_hat, Z_hat, omega_hat = mfa.obs2(Y,X,L,K,omega,fun,nb_steps,tempo)
    
    K = len(thetas_hat)
    tri_Y = ufc.tri(Y,omega_hat)
    tri_Z = ufc.tri(Z_hat,omega_hat)
    tri_X = ufc.tri(X,omega_hat)
    
    iotas_hat = np.zeros((K,D)).astype(int)
    reclus = ufc.tri((np.arange(N)).astype(int),omega_hat)
    Z_hac = np.zeros((N,L))
    thetas_hac = [[] for k in range(K)]
    
    for k in range(K):
        
        Y_k = tri_Y[k]
        Z_k = tri_Z[k]
        X_k = tri_X[k]
        card_k = len(Y_k)
        W_k_hat, V_k_hat, mu_k_hat, sigma_2_k_hat = thetas_hat[k]
        
        mu_k_hac = np.mean(Y_k,axis=0)
        mu_k_X = np.mean(X_k,axis=0)
        Xc_k = X_k - mu_k_X
        
        iota_k_hat = ufs.iotate(Y_k-Xc_k@np.transpose(V_k_hat),Z_k,U)
        iotas_hat[k] = iota_k_hat
        U = np.sum(iota_k_hat)
        
        Y_k_tilde = drr.discard(Y_k-Xc_k@np.transpose(V_k_hat),iota_k_hat)
        W_k_hac, Z_k_hac, sigma2_k_hac = sfa.PPCA(Y_k_tilde,L)
        
        for j in range(card_k):
            Z_hac[reclus[k][j]] = Z_k_hac[j]
        
        Y_k_hac = drr.FS_rec2(W_k_hac,Z_k_hac,V_k_hat,Xc_k,mu_k_hac,iota_k_hat)
        sigma2_k_hac = np.mean((Y_k-Y_k_hac)**2)
            
        thetas_hac[k] = [W_k_hac,V_k_hat,mu_k_hac,sigma2_k_hac]
        
    return thetas_hac, Z_hac, omega_hat, iotas_hat

def spe1(Y,L=None,K=None,omega=None,fun=clf.Lapras,tempo=True,latent=True,Dv=None,Lv=None):
    '''PPCA, puis clustering, puis Sélection de Variables.
    (Adapté pour le modèle (M.6.7))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    latent : bool, optional,
        Si mis sur False, le clustering se fait sur la vecteurs observés.
        Si mis sur True, le clustering se fait sur la vecteurs latents.
        Mis sur True par défaut.
    
    Dv : int, optional,
        Nombre de dimensions pertinentes de l'espace observé souhaité.
        Doit être inférieur ou égal à D.
        Mis sur None par défaut.
        Si mis sur None ou strictement supérieur à D, utilise la valeur de Dv renvoyée par la fonction U_opt.
    
    Lv : int, optional,
        Nombre de dimensions pertinentes de l'espace observé souhaité.
        Doit être inférieur ou égal à L.
        Mis sur None par défaut.
        Si mis sur None ou strictement supérieur à L, utilise la valeur de Lv renvoyée par la fonction U_opt.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas selon le cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des vecteurs observés.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas selon le cluster.
        
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents obtenus par la PPCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la loi des variables aléatoires diffère selon les clusters sur la dimension correspondante, et le nombre 0 signifie qu'elle ne diffère pas selon les clusters sur la dimension correspondante.
    '''
    N,D = np.shape(Y)    
    mu_hat = np.mean(Y,axis=0)
    
    if type(L) != type(None) and L >= D:
        print("L >= D")
        L = None
    
    if type(Dv) != type(None) and Dv > D:
        print("Dv > D")
        Dv = None
    
    if latent :
        
        W_prov, Z_prov, sigma2_hat = sfa.PPCA(Y,L)
        N,L = np.shape(Z_prov)
        
        D_W = np.diag(np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
        D_W_inv = np.diag(1/np.array([mbu.norme(W_prov[:,l]) for l in range(L)]))
        Z_hat = Z_prov @ D_W
        W_hat = W_prov @ D_W_inv
        
        if type(Lv) != type(None) and Lv > L:
            print("Lv > L")
            Lv = None
            
        if type(omega) == type(None):
            if type(K) == type(None):
                K = clf.K_opt(Z_hat)
            omega_hat = fun(Z_hat,K,tempo=tempo)
        else:
            omega_hat = omega
        
        iota_lat = ufs.iotate(Z_hat,omega_hat,Lv)
        Lv = np.sum(iota_lat)
        Lu = L-Lv
        tZu_hat,tZv_hat = ufc.tri(np.transpose(Z_hat),iota_lat)
        Zu_hat = np.transpose(tZu_hat)
        Zv_hat = np.transpose(tZv_hat)
        
        iota_hat = ufs.iotate_2(Y,Zv_hat,Dv)
        
    else:
        if type(omega) == type(None):
            if type(K) == type(None):
                K = clf.K_opt(Y)
            omega_hat = fun(Y,K,tempo=tempo)
        else:
            omega_hat = omega
        iota_hat = ufs.iotate(Y,omega_hat,Dv)
    
    Dv = np.sum(iota_hat)
    tYu,tYv = ufc.tri(np.transpose(Y),iota_hat)
    Yu = np.transpose(tYu)
    Yv = np.transpose(tYv)

    etav_hat, thetas_hat, Zv_hat, omega_hat = mfa.lat1(Yv,L=Lv,omega=omega_hat,tempo=tempo)
    Wv_hat = etav_hat[0]
    
    if type(L) != None and type(Lv) != None and L>=Lv:
        Lu = L - Lv
    else :
        Lu = None
    
    Wu_hat, Zu_hat, tau2_usef = sfa.PPCA(Yu,Lu)
    nu_hat, tau2_hat = sfa.MLE_Gauss(Zu_hat)
    
    eta_hac = [Wv_hat,Wu_hat,mu_hat,nu_hat,0.0,tau2_hat]
    Y_hat = drr.FS_sperec1(eta_hac,Zv_hat,Zu_hat,iota_hat)
    eta_hac[4] = np.mean((Y-Y_hat)**2)
    
    return eta_hac, thetas_hat, Zv_hat, Zu_hat, omega_hat, iota_hat

def spe2(Y,X,L,K=None,omega=None,fun=clf.Lapras,V=None,nb_steps=100,err=0.0,tempo=True,latent=True,Dv=None,Lv=None):
    '''RCA, puis clustering, puis Sélection de Variables.
    (Adapté pour le modèle (M.6.8))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont des vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int, optional,
        Nombre de dimensions latentes souhaité.
        Mis sur None par défaut.
        Si mis sur None, utilise la valeur de L renvoyée par la fonction L_opt.
    
    K : int, optional,
        Nombre de clusters à identifier.
        Si omega et K sont mis sur None, le nombre de clusters à identifier est estimé à partir de la fonction K_opt.
        Si omega est renseigné mais pas K, K prend la valuer du coefficient maximal de omega + 1.
        Mis sur None par défaut.
    
    omega : 1-D ndarray, optional,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
        Si mis sur None, omega est estimé à l'aide d'un algorithme de clustering.
        Mis sur None par défaut.
        
    fun : function, optional,
        Fonction de clustering à utiliser.
        Mis sur clf.Lapras() par défaut.
        
    tempo : bool, optional,
        Si mis sur False, ne sert à rien.
        Si mis sur True, affiche le nombre d'itérations effectuées par l'algorithme K-means.
        Mis sur True par défaut.
    
    latent : bool, optional,
        Si mis sur False, le clustering se fait sur la vecteurs observés.
        Si mis sur True, le clustering se fait sur la vecteurs latents.
        Mis sur True par défaut.
    
    Dv : int, optional,
        Nombre de dimensions pertinentes de l'espace observé souhaité.
        Doit être inférieur ou égal à D.
        Mis sur None par défaut.
        Si mis sur None ou strictement supérieur à D, utilise la valeur de Dv renvoyée par la fonction U_opt.
    
    Lv : int, optional,
        Nombre de dimensions pertinentes de l'espace observé souhaité.
        Doit être inférieur ou égal à L.
        Mis sur None par défaut.
        Si mis sur None ou strictement supérieur à L, utilise la valeur de Lv renvoyée par la fonction U_opt.
    
    Renvois
    -------
    eta : list,
        Liste de paramètres de la forme [Wv,Wu,V,mu,nu,sigma2,tau2], où :
            - Wv est une matrice de taille (Dv,Lv) dont les colonnes sont les axes principaux des dimensions "mixtes".
            - Wu est une matrice de taille (Du,Lu) dont les colonnes sont les axes principaux des dimensions "non-mixtes".
            - V est une matrice de taille (D,C) d'effets fixes.
            - mu est un vecteur de taille Dv+Du, supposé être la moyenne des vecteurs observés.
            - nu est un vecteur de taille Lu, supposé être la moyenne des vecteurs latents dont la loi ne change pas selon le cluster.
            - sigma2 est un réel positif, supposé être la variance du bruit des vecteurs observés.
            - tau2 est un réel positif, supposé être la variance des vecteurs latents dont la loi ne change pas selon le cluster.
        
    thetas : list,
        Liste de K éléments, dont chaque élément est une liste de paramètres de la forme [mu,sigma2] où :
            - mu est un vecteur de taille D, supposé être la moyenne des vecteurs du cluster.
            - sigma2 est un réel positif, supposé être la variance des vecteurs du cluster.
            
    Zv : 2-D ndarray,
        Matrice de taille (N,Lv) dont les lignes sont les vecteurs latents obtenus par la RCA.
    
    Zu : 2-D ndarray,
        Matrice de taille (N,Lu) dont les lignes sont les vecteurs latents obtenus par la RCA.
    
    omega : 1-D ndarray,
        Vecteur de taille N dont, pour tout n entre 0 et N-1, la n-ième valeur est un entier entre 0 et K-1 indiquant le numéro du cluster auquel appartient le n-ième individu.
    
    iota : 1-D ndarray,
        Vecteur de taille D contenant U fois le nombre 1 et D-U fois le nombre 0, où le nombre 1 signifie que la loi des variables aléatoires diffère selon les clusters sur la dimension correspondante, et le nombre 0 signifie qu'elle ne diffère pas selon les clusters sur la dimension correspondante.
    '''
    N,D = np.shape(Y)
    N1,C = np.shape(X)
    mu_hat = np.mean(Y,axis=0)
    Xc = X - np.mean(X,axis=0)
    
    if type(Dv) != type(None) and Dv > D:
        print("Dv > D")
        Dv = None
    
    if type(Lv) != type(None) and Lv > L:
            print("Lv > L")
            Lv = None
    
    if latent :
        
        W_hat, Z_hat, V_hat, sigma2_hat = sfa.ML_RCA(Y,X,L,V,nb_steps,err,tempo)
            
        if type(omega) == type(None):
            if type(K) == type(None):
                K = clf.K_opt(Z_hat)
            omega_hat = fun(Z_hat,K,tempo=tempo)
        else:
            omega_hat = omega
        
        iota_lat = ufs.iotate(Z_hat,omega_hat,Lv)
        Lv = np.sum(iota_lat)
        Lu = L-Lv
        tZu_hat,tZv_hat = ufc.tri(np.transpose(Z_hat),iota_lat)
        Zu_hat = np.transpose(tZu_hat)
        Zv_hat = np.transpose(tZv_hat)
        iota_hat = ufs.iotate_2(Y-Xc@np.transpose(V_hat),Zv_hat,Dv)
        
    else:
        if type(omega) == type(None):
            if type(K) == type(None):
                K = clf.K_opt(Y)
            omega_hat = fun(Y,K,tempo=tempo)
        else:
            omega_hat = omega
        iota_hat = ufs.iotate(Y,omega_hat,Dv)
    
    Dv = np.sum(iota_hat)
    tYu,tYv = ufc.tri(np.transpose(Y-Xc@np.transpose(V_hat)),iota_hat)
    Yu = np.transpose(tYu)
    Yv = np.transpose(tYv)

    etav_hat, thetas_hat, Zv_hat, omega_hat = mfa.lat1(Yv,L=Lv,omega=omega_hat,tempo=tempo)
    Wv_hat = etav_hat[0]
    
    N,Lv = np.shape(Zv_hat)
    Lu = L-Lv
    
    Wu_hat, Zu_hat, tau2_usef = sfa.PPCA(Yu,Lu)
    nu_hat, tau2_hat = sfa.MLE_Gauss(Zu_hat)
    
    eta_hac = [Wv_hat,Wu_hat,V_hat,mu_hat,nu_hat,0.0,tau2_hat]
    Y_hat = drr.FS_sperec2(eta_hac,Zv_hat,Zu_hat,X,iota_hat)
    eta_hac[5] = np.mean((Y-Y_hat)**2)
    
    return eta_hac, thetas_hat, Zv_hat, Zu_hat, omega_hat, iota_hat

NameError: name 'clf' is not defined

In [3]:
import pydoc as doc

In [17]:
pydoc?

In [16]:
doc.help(norme)

Help on function norme in module __main__:

norme(x)



# Tests

In [14]:
#Paramètres

N1 = 200
Dv1 = 12
Du1 = 8
D1 = Dv1+Du1
D1 = 20
Lv1 = 5
Lu1 = 3
L1 = Lv1+Lu1
L1 = 10
C1 = 4
K1 = 5
U1 = 8

K_tab = 2**np.arange(1,5)

eps = 1/10**6
la = 1.0
p = 0.1

In [15]:
eta,thetas = param_latmix_1(K1,D1,L1,s_glob3=0.5,s2=2.0,s3=0.8)

NameError: name 'mbu' is not defined

In [ ]:
W_hat,Z_hat,sigma2_hat = PPCA(Yc,L1)

In [ ]:
W,V,mu,sigma2 = sim_param_cov(D1,L1,C1,s1=2.0,s2=2.0,s4=0.5)
Z,Y,X = sim_data_cov(W,V,mu,sigma2,N1)
Sigma = X@np.transpose(X) + sigma2*np.eye(N1)
Yc = Y - mu

In [ ]:
W_hat,Z_hat,V_hat,Sigma_hat,sigma2_hat = ML_RCA(Y,X,L1,err=eps)

In [ ]:
XtV = X@np.transpose(V)
XtV_hat = X@np.transpose(V_hat)

for d in range(int(D1/2)):
    
    plt.figure()
    
    plt.scatter(XtV[:,2*d],XtV[:,2*d+1],label='$X.^t \check{V}$')
    plt.scatter(XtV_hat[:,2*d],XtV_hat[:,2*d+1],label='$X.^t \hat{V}$')
    
    for n in range(N1):
        plt.plot([XtV[n][2*d],XtV_hat[n][2*d]],[XtV[n][2*d+1],XtV_hat[n][2*d+1]],color='red')
    
    plt.legend()
    plt.show()

In [ ]:
ZtW = Z@np.transpose(W)
ZtW_hat = Z_hat@np.transpose(W_hat)

for d in range(int(D1/2)):
    
    plt.figure()
    
    plt.scatter(ZtW[:,2*d],ZtW[:,2*d+1],label='$Z.^tW$')
    plt.scatter(ZtW_hat[:,2*d],ZtW_hat[:,2*d+1],label='$Z.^t \hat{W}$')
    
    for n in range(N1):
        plt.plot([ZtW[n][2*d],ZtW_hat[n][2*d]],[ZtW[n][2*d+1],ZtW_hat[n][2*d+1]],color='red')
    
    plt.legend()
    plt.show()

In [ ]:

W_hat,Z_hat,V_hat,Sigma_hat,mu_hat = ML_RCA(Y,X,20)

In [ ]:
eta_hat, thetas_hat, Zv_hat, Zu_hat, omega_hat, iota_hat = FS_MFA_spe1(Y,L1,K1,fun=K_means_FPC,Dv=Dv1,Lv=Lv1)

In [ ]:
Y_hat = FS_sperec1(eta_hat,Zv_hat,Zu_hat,iota_hat)

In [ ]:
print(ARS(omega,omega_hat))

In [ ]:
print(iota)
print(iota_hat)

In [ ]:
MFA_graph(Y,Y_hat,omega_hat,omega=omega)

In [ ]:
#Cette fonction ne marche pas à cause d'un problème de rang
def EM_RCA(Y,X,L,V=None,Lambda=None,la=None,sigma2=None,nb_steps=1000,err=0.0,tempo=True):
    
    N1,D = np.shape(Y)
    N2,C = np.shape(X)
    
    if N1 != N2 :
        print('Erreur de dimensions sur Y et X')
        S, W, Z = PCA(Y,L)
        sigma2 = bruit(S,L)
        return W, Z, sigma2
    else :
        N = N1
    
    #Centrage de Y et X
    mY = np.mean(Y,axis=0)
    mX = np.mean(X,axis=0)
    Yc = np.array([y-mY for y in Y])
    Xc = np.array([x-mX for x in X])
    
    #Initialisation
    
    if type(V) != type(None) :
        
        #Copie de V s'il est donné
        V_hat = V.copy()
    
    #Estimation de V sinon
    else :
        #Initialisation
        S, W, Z = PCA(Yc,L)
        sigma2 = bruit(S,L)
        Sigma_hat = Xc@np.transpose(Xc) + sigma2 * np.eye(N)
        V_hat = V_estimator(Yc,Xc,Sigma_hat)
        
        dist = err+1
        t = 0
        
        #Boucle
        while dist > err and t < nb_steps :
            new_Sigma = Sigma_estimator(Yc,Xc,V_hat,L)
            new_V = V_estimator(Yc,Xc,new_Sigma)
            dist = np.sum((V_hat-new_V)**2) + np.sum((Sigma_hat-new_Sigma)**2)
            V_hat = new_V
            Sigma_hat = new_Sigma
            t += 1
        if tempo :
            print('t =',t)
    
    D1,C1 = np.shape(V_hat)
    if D != D1 or C != C1 :
        print('Erreur de dimensions sur V')
        S, W, Z = PCA(Yc,L)
        sigma2 = bruit(S,L)
        return W, Z, V_hat, np.eye(D), sigma2
    
    #Changement de variable
    X_tilde = Xc @ np.transpose(V_hat)
    
    #Le reste c'est la fonction du dessus
    W_hat,Z,Lambda_hat,sigma2 = EM_RCA_LRPSI(Yc,X_tilde,L,Lambda,la,sigma2,nb_steps,err,tempo)
    return W_hat,Z,V_hat,Lambda_hat,sigma2

In [18]:
help(np.sum)

Help on function sum in module numpy:

sum(a, axis=None, dtype=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)
    Sum of array elements over a given axis.
    
    Parameters
    ----------
    a : array_like
        Elements to sum.
    axis : None or int or tuple of ints, optional
        Axis or axes along which a sum is performed.  The default,
        axis=None, will sum all of the elements of the input array.  If
        axis is negative it counts from the last to the first axis.
    
        .. versionadded:: 1.7.0
    
        If axis is a tuple of ints, a sum is performed on all of the axes
        specified in the tuple instead of a single axis or all the axes as
        before.
    dtype : dtype, optional
        The type of the returned array and of the accumulator in which the
        elements are summed.  The dtype of `a` is used by default unless `a`
        has an integer dtype of less precision than the default platform
        integer.  In 

In [3]:
help(ML_RCA)

Help on function ML_RCA in module __main__:

ML_RCA(Y, X, L, V=None, nb_steps=100, err=0.0, tempo=True)
    Analyse en Composante Résiduelle, méthode itérative.
    (Adapté pour le modèle (M.2))
    
    Paramètres
    ----------
    Y : 2-D ndarray,
        Matrice de taille (N,D) dont les lignes sont les vecteurs observés.
    
    X : 2-D ndarray,
        Matrice de taille (N,C) dont les lignes sont des vecteurs de covariables.
        L'algorithme s'assure que les vecteurs de covariables sont centrés en les centrant de force.
    
    L : int,
        Nombre de dimensions latentes souhaité.
        
    V : 2-D ndarray, optional,
        Matrice de taille (D,C) d'effets fixes donnée en argument initial de l'algorithme itératif.
    
    nb_steps : int, optional,
        Nombre maximal d'itérations de l'algorithme.
        Mis sur 1000 par défaut.
        
    err : float, optional,
        Erreur en distance L2 entre deux solutions consécutives en-dessous de laquelle l'algorithme s

In [6]:
help(rd)

Help on package numpy.random in numpy:

NAME
    numpy.random

DESCRIPTION
    Random Number Generation
    
    Use ``default_rng()`` to create a `Generator` and call its methods.
    
    =============== =========================================================
    Generator
    --------------- ---------------------------------------------------------
    Generator       Class implementing all of the random number distributions
    default_rng     Default constructor for ``Generator``
    =============== =========================================================
    
    ============================================= ===
    BitGenerator Streams that work with Generator
    --------------------------------------------- ---
    MT19937
    PCG64
    Philox
    SFC64
    ============================================= ===
    
    ============================================= ===
    Getting entropy to initialize a BitGenerator
    --------------------------------------------- ---
    Seed